# MLOps AI Researcher

| | |
|-|-|
| Author(s) | [Keeyana Jones](https://github.com/keeyanajones/) |

## **A Notebook for MLOps AI Researchers**
An AI Researchers pre-configured Vertex AI Workbench Jupyter Notebook Offers a powerful, streamlined enviornment designed to accelerate research, experimentation, and rapid prototyping of advanced computer vision and multimodal AI solutions, particularly in the context of anomaly detection.  

Insted of spending weeks setting up infrastructure, researchers can immediately dive into their core work: designing and testing novel AI models.

#### 1. **Accelerated Experimentation and Prototyping**

- **Pre-configured Environment:**
Researchers get immediate access to JupyterLab with pre-installed TensorFlow and Python kernels, along with common ML/CV libraries (NumPy, Pandas, Scikit-learn, OpenCV, Pillow, etc). This eliminates environment setup headaches. 
- **Scalable Compute:**
Easy access to GPUs (e.g, NVIDIA V100, A100) directly from the workbench instance, crucial for training large vision models and runing computationally intensive experiments.
- **Direct Access to Data:**
Seamless integration with Cloud Storage and BigQuery allows researchers to quickly access and process large image/video datasets without complex authentication or data transfer steps.  
- **Version Control Integration (Git):**
Built-in Git functionality simplifes tracking code changes, collaborating with peers, and reverting to previous experiments.   

### 2. **Gemini Vision & Multimodal Capabilities:**

- **Gemini API Access:** Direct access to Gemini Vision APIs for multimodal understanding, allowing researchers to experiment with visual reasoning, image captioning, visual Q&A, and potentially leverage its capabilities for advanced anomaly detection feature extraction or contextual analysis.
- **Foundation Model Fine-tuning:** The environment supports fine-tuning of foundation models (including those powered by Gemini) on custom datasets, enabling researchers to adapt powerful pre-trained models to their specific anomaly detection tasks.

### 3. **MLOps Principles for Research Reproducibility:**

- **Containerization (Docker):** The ability to build and run Docker containers directly within the Workbench (especially with user-managed notebooks) means researchers can encapsulate their experimental environments, ensuring reproducibility and easier transition to production. This is vital for complex TensorFlow models.
- **Experiment Tracking:** Integration with Vertex AI Experiments allows researchers to automatically log model metrics, hyperparameters, and artifacts, enabling systematic comparison of different model architectures and training runs.
- **Reproducible Pipelines (KFP/Vertex AI Pipelines):** Although an MVP might simplify this, the foundation is there to transition successful experiments into reproducible pipelines, allowing researchers to test new ideas in a consistent, automated manner.

### 4. **Interactive Development with React (for Visualization & Interaction):**

- **Rapid UI Prototyping:** While not directly for core ML research, the inclusion of React allows AI researchers (or closely collaborating engineers) to quickly build interactive visualization tools for their model outputs. For anomaly detection, this means:
   - Visually inspecting detected anomalies (e.g., bounding boxes, heatmaps).
   - Testing human-in-the-loop feedback mechanisms.
   - Demonstrating the model's capabilities to non-technical stakeholders.
- **Real-time Model Interaction:** Researchers can use the React frontend to send real-time inference requests to deployed models, observing immediate results and fine-tuning their understanding of model behavior.

#### 5. **Simplified Deployment & Monitoring for Research Demos:**

- **One-Click Deployment (Simplified):** While full MLOps pipelines are for production, researchers can quickly deploy a prototype model to a Vertex AI Endpoint for testing purposes, making it accessible for team demos or integration testing.
- **Basic Monitoring:** Set up simple monitoring (e.g., request/latency metrics, basic prediction drift) to understand model behavior in a "pseudo-production" environment, helping identify early issues during research.

----

## Pre-configured Enviornment 

### Install Google and other required packages

In [ ]:
! pip install --upgrade pip \
google-cloud-aiplatform \
google-cloud-storage \
google-generativeai \
google-cloud-bigquery \
google-cloud-logging \
google-cloud-monitoring 

In [ ]:
! pip install --upgrade pip \
tensorflow \
torch torchvision \
scikit-learn pandas numpy matplotlib seaborn plotly bokeh \
xgboost lightgbm \
tqdm \
pillow opencv-python h5py \
requests beautifulsoup4 lxml \
nltk spacy \
jupyter jupyterlab jupyterlab-git nbformat ipywidgets

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import os
from google import genai

PROJECT_ID = "[your-project-id]"  
# @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries

In [ ]:
# Import the necessary functions/classes
from IPython.display import Markdown, display

### Load Models

In [ ]:
MODEL_ID = "gemini-2.0-flash-001"  # @param {type:"string"}

----

# Getting Started

## Tasks for this Environment

### **Data Acquisition & Curation for Vision Models:**

- **Loading and Exploring Datasets:** Use Python (Pandas, NumPy, Matplotlib, Seaborn) in Jupyter notebooks to load image/video data from Cloud Storage, explore its distribution, and visualize samples.
- **Data Augmentation:** Apply and test various image augmentation techniques (e.g., rotations, flips, color jitter, noise injection) using libraries like TensorFlow Data, Albumentations, or OpenCV.
- **Data Labeling (Anomaly vs. Normal):** Configure and manage Vertex AI Data Labeling jobs (using templates/data_labeling_job.yaml) to get human annotations for anomaly types or normal patterns in images.
- **Dataset Versioning:** Integrate with Vertex AI Datasets to manage and version different iterations of their vision datasets.

### Loading and Exploring Datasets:

- Data Access: Connect to Cloud Storage buckets (e.g., for raw image data, processed features) and BigQuery tables.

**Setup and Authentication**

In [ ]:
# Import necessary libraries
import os
import io
import json
import random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Google Cloud SDKs
from google.cloud import storage
from google.cloud import aiplatform
from google.cloud import vision_v1
from google.cloud.vision_v1 import types
from google.protobuf import json_format

# TensorFlow (for tf.data and image processing)
import tensorflow as tf
import tensorflow_datasets as tfds

# Image Augmentation Libraries
import cv2 # OpenCV
import albumentations as A # Albumentations

# Set up GCP project and region
# Ensure these are configured correctly for your environment
# Or, they can be dynamically fetched if the service account has permissions.
PROJECT_ID = os.getenv('GOOGLE_CLOUD_PROJECT')
REGION = 'us-central1' # Or your desired region, e.g., 'europe-west4'

if not PROJECT_ID:
    # Fallback if GOOGLE_CLOUD_PROJECT env var is not set
    # You might need to authenticate `gcloud auth application-default login`
    # if running locally, or rely on instance metadata if on a VM.
    try:
        _, project_id = aiplatform.initializer.global_config.get_client_options()
        PROJECT_ID = project_id
    except Exception:
        print("Warning: GOOGLE_CLOUD_PROJECT environment variable not set. Please set it or ensure service account is configured.")
        PROJECT_ID = "YOUR_GCP_PROJECT_ID" # REPLACE WITH YOUR PROJECT ID

print(f"Using Google Cloud Project: {PROJECT_ID}")
print(f"Using Google Cloud Region: {REGION}")

# Initialize Vertex AI SDK
aiplatform.init(project=PROJECT_ID, location=REGION)

# Initialize Google Cloud Storage client
gcs_client = storage.Client(project=PROJECT_ID)

# Initialize Google Cloud Vision API client
vision_client = vision_v1.ImageAnnotatorClient()

print("Setup complete. Libraries imported and GCP clients initialized.")

# Optional: Configure matplotlib for better visualization
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 7)
plt.rcParams['figure.dpi'] = 100
sns.set_style('whitegrid')

In [ ]:
# Loading and Exploring Datasets from Cloud Storage

# --- Configuration for your dataset ---
GCS_BUCKET_NAME = 'your-image-dataset-bucket' # REPLACE with your actual GCS bucket name
GCS_DATA_PREFIX = 'raw_images/' # Optional: if your images are in a subfolder, e.g., 'raw_images/'
IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
MAX_SAMPLES_TO_LIST = 1000 # Limit for listing all files to avoid huge memory usage
SAMPLES_TO_LOAD = 5 # Number of random images to load and display

print(f"Attempting to connect to GCS bucket: gs://{GCS_BUCKET_NAME}/{GCS_DATA_PREFIX}")

try:
    bucket = gcs_client.get_bucket(GCS_BUCKET_NAME)
    blobs = list(bucket.list_blobs(prefix=GCS_DATA_PREFIX))
    
    image_blobs = [b for b in blobs if b.name.lower().endswith(IMAGE_EXTENSIONS)]
    
    if not image_blobs:
        print(f"No image files found in gs://{GCS_BUCKET_NAME}/{GCS_DATA_PREFIX}. Please check bucket name and prefix.")
    else:
        print(f"Found {len(image_blobs)} image files.")

        # --- Explore Dataset Distribution (if metadata is available) ---
        # For simple image datasets, distribution might be based on file size or creation date.
        # If you have a CSV metadata file in GCS, you'd load it here.
        # Example: loading a dummy metadata file for demonstration
        try:
            metadata_blob = bucket.blob(f'{GCS_DATA_PREFIX}metadata.csv') # Replace with your actual metadata file
            if metadata_blob.exists():
                metadata_df = pd.read_csv(io.BytesIO(metadata_blob.download_as_bytes()))
                print(f"\nLoaded metadata for {len(metadata_df)} images.")
                print("Metadata head:")
                print(metadata_df.head())
                
                # Example: Visualize distribution of a 'label' column if it exists
                if 'label' in metadata_df.columns:
                    plt.figure(figsize=(8, 5))
                    sns.countplot(data=metadata_df, x='label')
                    plt.title('Distribution of Image Labels (from Metadata)')
                    plt.xlabel('Label')
                    plt.ylabel('Count')
                    plt.xticks(rotation=45)
                    plt.tight_layout()
                    plt.show()
                else:
                    print("No 'label' column found in metadata for distribution visualization.")

            else:
                print("No metadata.csv found in bucket. Skipping metadata exploration.")
        except Exception as e:
            print(f"Error loading or processing metadata: {e}")

        # --- Visualize Sample Images ---
        print(f"\nDisplaying {SAMPLES_TO_LOAD} random image samples:")
        sample_blobs = random.sample(image_blobs, min(SAMPLES_TO_LOAD, len(image_blobs)))

        plt.figure(figsize=(15, 5 * ((SAMPLES_TO_LOAD + 4) // 5))) # Adjust grid based on sample count
        for i, blob in enumerate(sample_blobs):
            try:
                img_bytes = blob.download_as_bytes()
                img = Image.open(io.BytesIO(img_bytes))
                
                plt.subplot(((SAMPLES_TO_LOAD + 4) // 5), 5, i + 1)
                plt.imshow(img)
                plt.title(f"{blob.name.split('/')[-1]}")
                plt.axis('off')
            except Exception as e:
                print(f"Could not load image {blob.name}: {e}")
        plt.tight_layout()
        plt.show()

except Exception as e:
    print(f"Error accessing GCS bucket {GCS_BUCKET_NAME}: {e}")
    print("Please ensure the bucket exists and your service account has 'Storage Object Viewer' role.")

### Data Augmentation:

In [ ]:
# Data Augmentation
print("Demonstrating Image Augmentation Techniques.")

# --- Helper function to load an image ---
def load_image_from_gcs(gcs_uri, target_size=(256, 256)):
    """Loads an image from GCS and resizes it."""
    try:
        bucket_name = gcs_uri.split('gs://')[1].split('/')[0]
        blob_name = '/'.join(gcs_uri.split('gs://')[1].split('/')[1:])
        bucket = gcs_client.get_bucket(bucket_name)
        blob = bucket.blob(blob_name)
        img_bytes = blob.download_as_bytes()
        img = Image.open(io.BytesIO(img_bytes)).convert("RGB") # Ensure RGB
        return np.array(img.resize(target_size))
    except Exception as e:
        print(f"Error loading image {gcs_uri}: {e}")
        return None

# Choose a sample image for augmentation demonstration
if image_blobs:
    sample_image_blob = random.choice(image_blobs)
    sample_gcs_uri = f"gs://{sample_image_blob.bucket.name}/{sample_image_blob.name}"
    original_image_np = load_image_from_gcs(sample_gcs_uri)
else:
    print("No images found in GCS bucket to demonstrate augmentation. Creating a dummy image.")
    original_image_np = np.zeros((256, 256, 3), dtype=np.uint8)
    cv2.putText(original_image_np, "No Image", (50, 128), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)


if original_image_np is not None:
    plt.figure(figsize=(18, 6))

    # --- 1. Original Image ---
    plt.subplot(1, 4, 1)
    plt.imshow(original_image_np)
    plt.title('Original Image')
    plt.axis('off')

    # --- 2. Albumentations Example ---
    # Define an augmentation pipeline
    augmentation_pipeline_alb = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.GaussNoise(p=0.2),
        A.CoarseDropout(max_holes=8, max_height=8, max_width=8, p=0.2),
    ])
    augmented_image_alb = augmentation_pipeline_alb(image=original_image_np)['image']
    plt.subplot(1, 4, 2)
    plt.imshow(augmented_image_alb)
    plt.title('Albumentations Augmented')
    plt.axis('off')

    # --- 3. TensorFlow Data Augmentation Example ---
    # Convert to TensorFlow tensor
    tf_image = tf.convert_to_tensor(original_image_np)

    # Apply simple TF augmentations
    tf_augmented_image = tf.image.random_flip_left_right(tf_image)
    tf_augmented_image = tf.image.random_brightness(tf_augmented_image, max_delta=0.2)
    tf_augmented_image = tf.image.random_contrast(tf_augmented_image, lower=0.8, upper=1.2)
    
    # Optional: Rotate (needs custom TF ops or tf_addons for more advanced)
    # tf_augmented_image = tf.image.rot90(tf_augmented_image, k=1) # 90-degree rotation

    plt.subplot(1, 4, 3)
    plt.imshow(tf_augmented_image.numpy().astype(np.uint8)) # Convert back to numpy for display
    plt.title('TensorFlow Augmented')
    plt.axis('off')

    # --- 4. OpenCV Example (Simple Rotation) ---
    angle = 30 # degrees
    (h, w) = original_image_np.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image_cv = cv2.warpAffine(original_image_np, M, (w, h))

    plt.subplot(1, 4, 4)
    plt.imshow(rotated_image_cv)
    plt.title(f'OpenCV Rotated ({angle}°)')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    print("\nTip: For real-world use, integrate augmentation directly into your TensorFlow/PyTorch Dataset pipelines.")
else:
    print("Skipping augmentation demonstration as no image was loaded.")

### Data Labeling (Anomaly vs. Normal):

In [ ]:
# Data Labeling (Anomaly vs. Normal) with Vertex AI Data Labeling

print("Demonstrating how to initiate a Vertex AI Data Labeling Job.")
print("The actual labeling is done in the Vertex AI Console by human annotators.")

# --- Configuration for your labeling job ---
# Input GCS URI of a CSV or JSONL file pointing to your images.
# Each line in the file should be a GCS URI to an image, e.g., 'gs://your-image-dataset-bucket/raw_images/img_001.jpg'
INPUT_DATA_GCS_URI = f'gs://{GCS_BUCKET_NAME}/data_for_labeling.csv' # REPLACE with your input data list

# Output GCS URI for the labeled data
OUTPUT_GCS_URI = f'gs://{GCS_BUCKET_NAME}/labeled_data_output/' # Labeled data will be written here

# Display name for your labeling job
LABELING_JOB_DISPLAY_NAME = 'Anomaly_Detection_Labeling_Job_V1'

# Instruction URI for labelers (a PDF or web page with clear labeling guidelines)
INSTRUCTION_URI = 'gs://cloud-samples-data/ai-platform-unified/instructions/instructions.pdf' # REPLACE or create your own

# Annotation spec schema URI (JSON schema for your labels)
# For 'Anomaly vs. Normal' you might have a simple classification schema.
# Example: a schema for image classification where 'anomaly' and 'normal' are labels.
# Example content for `classification_schema.yaml` in GCS:
# data_item_schema_uri: "gs://google-cloud-aiplatform/schema/data_item/image_1.0.0.yaml"
# annotation_types:
#   - display_name: "Anomaly"
#     description: "The image shows an anomaly."
#     label_type: "classification"
#   - display_name: "Normal"
#     description: "The image shows a normal pattern."
#     label_type: "classification"
ANNOTATION_SPEC_SCHEMA_URI = f'gs://{GCS_BUCKET_NAME}/labeling_schemas/anomaly_normal_classification_schema.yaml' # REPLACE with your actual schema

# Worker pool configuration (optional, typically used for custom workforce)
# For Google-managed workforce, leave this as default.
# See: https://cloud.google.com/vertex-ai/docs/reference/rest/v1/projects.locations.dataLabelingJobs

# Duration for which the labeling job is available for annotators
# 7 days (604800 seconds)
LABELING_JOB_DURATION_SECONDS = 604800

# Number of human annotators per data item (e.g., 3 for higher confidence)
ANNOTATOR_COUNT = 3

print(f"\nLabeling Job Configuration:")
print(f"  Input Data: {INPUT_DATA_GCS_URI}")
print(f"  Output URI: {OUTPUT_GCS_URI}")
print(f"  Job Name: {LABELING_JOB_DISPLAY_NAME}")
print(f"  Instructions: {INSTRUCTION_URI}")
print(f"  Schema: {ANNOTATION_SPEC_SCHEMA_URI}")
print(f"  Annotators per item: {ANNOTATOR_COUNT}")


# --- Step 1: Prepare your input data file (if not already done) ---
# Create a dummy CSV with GCS URIs for demonstration
# In a real scenario, this file would list all images you want to label.
if not gcs_client.get_bucket(GCS_BUCKET_NAME).blob(INPUT_DATA_GCS_URI.split(f'{GCS_BUCKET_NAME}/')[1]).exists():
    print(f"\nCreating a dummy input data file: {INPUT_DATA_GCS_URI}")
    # Get a few sample image URIs to put into the CSV
    sample_uris = [f"gs://{b.bucket.name}/{b.name}" for b in random.sample(image_blobs, min(10, len(image_blobs)))]
    dummy_csv_content = "\n".join(sample_uris)
    
    input_blob = gcs_client.get_bucket(GCS_BUCKET_NAME).blob(INPUT_DATA_GCS_URI.split(f'{GCS_BUCKET_NAME}/')[1])
    input_blob.upload_from_string(dummy_csv_content, content_type='text/csv')
    print(f"Dummy input data created at {INPUT_DATA_GCS_URI}")
else:
    print(f"Input data file already exists at {INPUT_DATA_GCS_URI}")


# --- Step 2: Ensure your annotation schema is in GCS ---
# You would upload your actual `anomaly_normal_classification_schema.yaml` here.
# For demonstration, let's create a very basic placeholder if it doesn't exist.
if not gcs_client.get_bucket(GCS_BUCKET_NAME).blob(ANNOTATION_SPEC_SCHEMA_URI.split(f'{GCS_BUCKET_NAME}/')[1]).exists():
    print(f"\nCreating a dummy annotation schema at {ANNOTATION_SPEC_SCHEMA_URI}")
    dummy_schema_content = """
data_item_schema_uri: "gs://google-cloud-aiplatform/schema/data_item/image_1.0.0.yaml"
annotation_types:
  - display_name: "Anomaly"
    description: "The image shows an anomaly."
    label_type: "classification"
  - display_name: "Normal"
    description: "The image shows a normal pattern."
    label_type: "classification"
"""
    schema_blob = gcs_client.get_bucket(GCS_BUCKET_NAME).blob(ANNOTATION_SPEC_SCHEMA_URI.split(f'{GCS_BUCKET_NAME}/')[1])
    schema_blob.upload_from_string(dummy_schema_content, content_type='text/yaml')
    print(f"Dummy schema created at {ANNOTATION_SPEC_SCHEMA_URI}")
else:
    print(f"Annotation schema already exists at {ANNOTATION_SPEC_SCHEMA_URI}")

# --- Step 3: Create and Run the Data Labeling Job ---
try:
    # Use the `aiplatform.ImageDataset.create_from_gcs()` method
    # It will implicitly create a dataset and then start the labeling job.
    # Note: For `DataLabelingJob` directly, you'd typically define a `model_evaluation_job` and then create the job.
    # The SDK provides a simpler way to start an image labeling job directly.
    
    # We first create a dummy Vertex AI Dataset. Data Labeling Job needs a Dataset ID.
    # In a real scenario, you'd create this Dataset and then pass its ID.
    # Or, the labeling job can create a temporary dataset.
    
    # The `aiplatform.DataLabelingJob.create()` method is the most direct way
    # to initiate from Python.
    
    print("\nAttempting to create Vertex AI Data Labeling Job...")

    # The actual job creation payload
    data_labeling_job_payload = {
        "display_name": LABELING_JOB_DISPLAY_NAME,
        "specialist_pools": [], # Use Google-managed workforce
        "input_config": {
            "gcs_source": {
                "input_uris": [INPUT_DATA_GCS_URI]
            },
            "data_type": "IMAGE" # For image labeling
        },
        "output_config": {
            "gcs_destination": {
                "output_uri_prefix": OUTPUT_GCS_URI
            }
        },
        "instruction_uri": INSTRUCTION_URI,
        "annotation_spec_set_config": {
            "annotation_spec_schema_uri": ANNOTATION_SPEC_SCHEMA_URI,
            "replica_count": ANNOTATOR_COUNT,
            "question_type": "IMAGE_CLASSIFICATION", # Or IMAGE_BOUNDING_BOX, IMAGE_SEGMENTATION etc.
            "human_annotation_management_config": {
                "human_annotation_request_config": {
                    "question_type": "IMAGE_CLASSIFICATION_ANNOTATION",
                    "instruction_uri": INSTRUCTION_URI,
                    "manual_batch_tuning_parameters": {},
                    "schema_configs": [
                        # This part specifies how your schema maps to the UI
                        # For simple classification, this is often handled implicitly
                    ],
                }
            }
        },
        "duration": {"seconds": LABELING_JOB_DURATION_SECONDS}
    }

    # Use the low-level client for full flexibility if needed, or the SDK
    # For now, let's use the SDK's higher-level wrapper if available for this specific task
    # Or, show the REST API approach for clarity if SDK doesn't abstract it well.

    # Option 1: Use SDK's `create_image_data_labeling_job` (more direct for common cases)
    # This might require some adjustments to parameters based on the latest SDK version.
    
    # Option 2: Use the lower-level `aiplatform.gapic.JobServiceClient` for full control
    from google.cloud.aiplatform_v1.services import job_service
    from google.cloud.aiplatform_v1.types import DataLabelingJob, GcsSource, GcsDestination

    job_client = job_service.JobServiceClient(client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"})

    # Prepare the DataLabelingJob object
    dl_job = DataLabelingJob(
        display_name=LABELING_JOB_DISPLAY_NAME,
        inputs=GcsSource(input_uris=[INPUT_DATA_GCS_URI]),
        outputs=GcsDestination(output_uri_prefix=OUTPUT_GCS_URI),
        # You'll need to define the instructions and annotation spec set directly in the proto format
        # This is where the complexity comes in for `DataLabelingJob` via client.
        # For simplicity, if SDK offers a direct method for Image Classification Labeling, use that.
        
        # Example using the higher-level SDK for image classification
        # Note: This is an example, please refer to latest SDK docs for exact method signature
        # as it can vary.
        # aiplatform.ImageDataset.create_from_gcs requires an import path not provided directly
        # by the basic aiplatform client.
    )

    # Simplified call via `aiplatform` SDK if it supports it cleanly:
    # This is a conceptual call; the actual method signature might differ.
    # Refer to `aiplatform.DataLabelingJob` documentation.
    
    # As of current SDK, creating directly is complex. It's usually done via console
    # or by wrapping it in a custom Python function that builds the proto.
    print("\nNote: Directly creating complex Data Labeling Jobs via Python SDK can be verbose.")
    print("For a full functional example, refer to Google Cloud's official documentation for Data Labeling API.")
    print(f"You would typically navigate to Vertex AI -> Data Labeling in the console to create/monitor the job, or use a more detailed Python script.")
    print(f"Your input data list is: {INPUT_DATA_GCS_URI}")
    print(f"Your schema is at: {ANNOTATION_SPEC_SCHEMA_URI}")
    print(f"Once created, monitor the job at: https://console.cloud.google.com/vertex-ai/data-labeling-jobs/list?project={PROJECT_ID}&region={REGION}")

    # Dummy placeholder for actually starting the job (replace with real code if needed)
    # data_labeling_job = aiplatform.DataLabelingJob.create(
    #     display_name=LABELING_JOB_DISPLAY_NAME,
    #     # ... other parameters from data_labeling_job_payload
    # )
    # print(f"Data Labeling Job '{data_labeling_job.display_name}' created. ID: {data_labeling_job.name}")
    # print(f"Monitor job in console: {data_labeling_job.resource_name}")

except Exception as e:
    print(f"Error creating Data Labeling Job: {e}")
    print("Please ensure your service account has 'Vertex AI User' and 'Data Labeler' roles, and necessary files exist in GCS.")
    print("Also ensure the Vertex AI Data Labeling API is enabled in your project.")

### Dataset Versioning:

In [ ]:
# Dataset Versioning with Vertex AI Datasets

print("Demonstrating Dataset Versioning with Vertex AI Datasets.")

# --- Configuration ---
DATASET_DISPLAY_NAME = 'Anomaly_Vision_Dataset'
DATASET_GCS_INPUT_URI = f'gs://{GCS_BUCKET_NAME}/{GCS_DATA_PREFIX}' # Your source image directory
# If you have a specific CSV/JSONL file that lists all images in the dataset, use that:
# DATASET_GCS_INPUT_URI = f'gs://{GCS_BUCKET_NAME}/all_image_uris.csv'

# --- 1. Create a new Vertex AI Dataset (if it doesn't exist) ---
print(f"\nChecking for existing dataset: {DATASET_DISPLAY_NAME}")
datasets = aiplatform.ImageDataset.list(filter=f'display_name="{DATASET_DISPLAY_NAME}"')

if datasets:
    dataset = datasets[0]
    print(f"Found existing dataset: {dataset.display_name} (ID: {dataset.resource_name})")
    print("You can typically import new data to update a dataset, which implicitly creates a new version.")
else:
    print(f"Creating new dataset: {DATASET_DISPLAY_NAME}")
    try:
        # Create an Image Dataset
        # The import process can take time.
        # For large datasets, consider creating an empty dataset first, then importing in chunks.
        dataset = aiplatform.ImageDataset.create(
            display_name=DATASET_DISPLAY_NAME,
            gcs_source=DATASET_GCS_INPUT_URI, # Path to your images (can be folder or URI list)
            import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification, # Or image.bounding_box etc.
            sync=True # Wait for the dataset creation to complete
        )
        print(f"Dataset '{dataset.display_name}' created successfully. ID: {dataset.resource_name}")
        print(f"Dataset in console: https://console.cloud.google.com/vertex-ai/datasets/images/{dataset.name.split('/')[-1]}?project={PROJECT_ID}&region={REGION}")

    except Exception as e:
        print(f"Error creating dataset: {e}")
        print("Ensure the GCS_DATA_PREFIX contains valid image files and your service account has necessary permissions.")
        print("You may need to provide a CSV/JSONL file with image URIs if direct folder import doesn't work for your setup.")
        # Attempt to list datasets even if creation failed, to prevent further errors
        datasets = aiplatform.ImageDataset.list(filter=f'display_name="{DATASET_DISPLAY_NAME}"')
        if datasets:
            dataset = datasets[0]
            print(f"Found existing dataset despite creation error: {dataset.display_name}")
        else:
            print("Could not find or create dataset. Skipping further steps.")
            dataset = None # Ensure dataset is None if not found/created


if dataset:
    # --- 2. Check Dataset Versioning / Import New Data ---
    print(f"\nDataset ID: {dataset.resource_name}")
    print(f"Dataset labels: {dataset.labels}") # Labels can be used for versioning/metadata

    # To create a new "version" or update the dataset, you import new data.
    # Vertex AI Datasets manages versions implicitly by tracking the import history.
    # Each import job creates a new "snapshot" or iteration of the dataset.

    # Example: If you have a new set of labeled data (e.g., from your Data Labeling Job)
    # You would import it into this dataset.
    LABELED_DATA_IMPORT_URI = f'gs://{GCS_BUCKET_NAME}/labeled_data_output/annotations.jsonl' # Example from Data Labeling Job output

    print(f"\nAttempting to import new data (simulating a new version/update).")
    print(f"Importing from: {LABELED_DATA_IMPORT_URI}")
    
    try:
        # This will create a new import job for the dataset.
        # It's important to have a valid import schema for your data (e.g., for labeled images).
        # For labeled data, the schema will be different (e.g., `single_label_classification`).
        # This process implicitly versions the dataset.
        
        # NOTE: A real import requires `LABELED_DATA_IMPORT_URI` to be a valid JSONL
        # file in the Vertex AI Dataset import format, e.g., with image URIs and annotations.
        # Example structure for image classification JSONL:
        # {"imageGcsUri": "gs://bucket/img1.jpg", "classificationAnnotation": {"displayName": "Normal"}}
        # {"imageGcsUri": "gs://bucket/img2.jpg", "classificationAnnotation": {"displayName": "Anomaly"}}

        # Check if dummy labeled data exists (from Cell 4)
        if not gcs_client.get_bucket(GCS_BUCKET_NAME).blob(LABELED_DATA_IMPORT_URI.split(f'{GCS_BUCKET_NAME}/')[1]).exists():
             print(f"Warning: Dummy labeled data file not found at {LABELED_DATA_IMPORT_URI}. Skipping import.")
             print("Please run Cell 4 to create a labeling job first, then generate some dummy output.")
             print("Alternatively, create a dummy annotations.jsonl manually with valid format.")
        else:
            # This is a blocking call: `sync=True`
            dataset.import_data(
                gcs_source=[LABELED_DATA_IMPORT_URI],
                import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification, # Adjust schema as needed
                sync=True
            )
            print("New data successfully imported into the dataset. This acts as a new 'version'.")
            print("The dataset now includes the newly imported data, implicitly versioned by the import job.")
            print(f"You can see import history in the Vertex AI console under the dataset's 'Imports' tab.")

    except Exception as e:
        print(f"Error importing new data into dataset: {e}")
        print("Ensure the import URI points to a valid file formatted according to the schema.")
        print("Common issues: incorrect JSONL format, invalid image URIs, schema mismatch.")

    # --- 3. Accessing Dataset Information (for MLOps Tracking) ---
    print(f"\nDataset resource name: {dataset.resource_name}")
    print(f"Dataset create time: {dataset.create_time.isoformat()}")
    print(f"Dataset update time: {dataset.update_time.isoformat()}")

    # You can get the import history, which serves as version tracking
    print("\nRecent Dataset Import History (implicit versioning):")
    # This requires using the lower-level GAPIC client for detailed history or parsing logs.
    # The `aiplatform` SDK often provides simpler ways to get core info.
    # For a full history, you'd typically look at `dataset.metadata_artifact.lineage_subgraph` or monitor operations.
    print("Please view the full import history in the Vertex AI console under the dataset's 'Imports' tab.")


    # --- 4. Using the Dataset for Training ---
    print("\nThis dataset can now be used as input for Vertex AI Training jobs.")
    print("Example (conceptual):")
    print("training_job = aiplatform.CustomTrainingJob(...)")
    print("model = training_job.run(dataset=dataset, ...)")

else:
    print("Skipping dataset versioning steps as no dataset could be created or found.")

### **Anomaly Detection Model Development & Experimentation:**

- **Architectural Exploration:** Experiment with various TensorFlow model architectures for anomaly detection (e.g., autoencoders, GANs for anomaly generation, one-class classification models, pre-trained backbone + anomaly head).
- **Gemini Vision Integration:** Experiment with using Gemini Vision for generating image embeddings, classifying image content, or generating descriptions that could aid anomaly detection (e.g., "damaged part of machine").
- **Hyperparameter Tuning:** Use Vertex AI Vizier to systematically tune hyperparameters for their anomaly detection models, leveraging Workbench's compute power.
- **Custom Training Logic:** Write and test custom TensorFlow training loops and loss functions specifically tailored for anomaly detection (e.g., reconstruction loss, novelty detection metrics).
- **Experiment Tracking:** Log model performance metrics (e.g., AUC, precision, recall for anomaly detection), training curves, and model checkpoints to Vertex AI Experiments.

### Loading and Exploring Datasets:

- Data Access: Connect to Cloud Storage buckets (e.g., for raw image data, processed features) and BigQuery tables.

In [ ]:
# Setup and Basic Data Loading for Model Development

import os
import io
import random
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Google Cloud SDKs
from google.cloud import storage
from google.cloud import aiplatform
from google.cloud.vision_v1 import types
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part, Image as GImage

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, models, optimizers

# Scikit-learn for evaluation metrics
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
from sklearn.model_selection import train_test_split

# Set up GCP project and region (ensure these match your environment)
PROJECT_ID = os.getenv('GOOGLE_CLOUD_PROJECT')
REGION = 'us-central1' # Or your desired region

if not PROJECT_ID:
    try:
        _, project_id = aiplatform.initializer.global_config.get_client_options()
        PROJECT_ID = project_id
    except Exception:
        PROJECT_ID = "YOUR_GCP_PROJECT_ID" # REPLACE WITH YOUR PROJECT ID

print(f"Using Google Cloud Project: {PROJECT_ID}")
print(f"Using Google Cloud Region: {REGION}")

# Initialize Vertex AI SDK
aiplatform.init(project=PROJECT_ID, location=REGION)

# Initialize Google Cloud Storage client
gcs_client = storage.Client(project=PROJECT_ID)

# Initialize Vertex AI for Generative AI (Gemini)
vertexai.init(project=PROJECT_ID, location=REGION)
gemini_model_pro_vision = GenerativeModel("gemini-pro-vision")
gemini_model_pro = GenerativeModel("gemini-pro") # For text-only generation

# --- Configuration for your dataset (use your "normal" data for training anomaly models) ---
# Assuming your 'normal' images are in a specific GCS subfolder
GCS_NORMAL_DATA_PREFIX = 'raw_images/normal/' # REPLACE with your actual normal data path
GCS_ANOMALY_DATA_PREFIX = 'raw_images/anomaly/' # REPLACE with your actual anomaly data path (for testing)
GCS_BUCKET_NAME = 'your-image-dataset-bucket' # REPLACE with your actual GCS bucket name

IMAGE_SIZE = (128, 128) # Standardize image size for models
BATCH_SIZE = 32

print(f"\nLoading sample 'normal' images from: gs://{GCS_BUCKET_NAME}/{GCS_NORMAL_DATA_PREFIX}")

# --- Helper function to load and preprocess images for TensorFlow ---
def load_and_preprocess_image(gcs_uri, label=None, image_size=IMAGE_SIZE):
    """Loads image from GCS URI, decodes, resizes, and normalizes."""
    img_bytes = tf.io.read_file(gcs_uri)
    img = tf.image.decode_jpeg(img_bytes, channels=3) # Adjust for PNG/BMP as needed
    img = tf.image.resize(img, image_size)
    img = img / 255.0 # Normalize to [0, 1]
    if label is not None:
        return img, label
    return img

# --- Load a small sample of normal image URIs for in-notebook experimentation ---
normal_image_uris = []
try:
    bucket = gcs_client.get_bucket(GCS_BUCKET_NAME)
    normal_blobs = list(bucket.list_blobs(prefix=GCS_NORMAL_DATA_PREFIX))
    normal_image_uris = [f"gs://{b.bucket.name}/{b.name}" for b in normal_blobs if b.name.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Found {len(normal_image_uris)} 'normal' images.")

    # Create a TensorFlow Dataset for normal data
    # Use a small subset for in-notebook quick tests
    num_normal_samples = min(500, len(normal_image_uris)) # Limit for quick testing
    train_uris, val_uris = train_test_split(normal_image_uris[:num_normal_samples], test_size=0.2, random_state=42)

    train_ds = tf.data.Dataset.from_tensor_slices(train_uris).map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
    val_ds = tf.data.Dataset.from_tensor_slices(val_uris).map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

    print(f"Prepared {len(train_uris)} training normal samples and {len(val_uris)} validation normal samples.")

    # Load some anomaly image URIs for testing
    anomaly_image_uris = []
    anomaly_blobs = list(bucket.list_blobs(prefix=GCS_ANOMALY_DATA_PREFIX))
    anomaly_image_uris = [f"gs://{b.bucket.name}/{b.name}" for b in anomaly_blobs if b.name.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Found {len(anomaly_image_uris)} 'anomaly' images for testing.")

    # Create a TensorFlow Dataset for anomaly data (for evaluation later)
    anomaly_ds = tf.data.Dataset.from_tensor_slices(anomaly_image_uris).map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

except Exception as e:
    print(f"Error loading initial image data: {e}")
    print("Ensure GCS_BUCKET_NAME, GCS_NORMAL_DATA_PREFIX, and GCS_ANOMALY_DATA_PREFIX are correct.")
    train_ds, val_ds, anomaly_ds = None, None, None # Prevent further errors if data not loaded

print("Setup complete. Basic data loaded.")

#### Architectural Exploration:

In [ ]:
# Architectural Exploration (Autoencoders for Anomaly Detection)

print("--- Architectural Exploration: Autoencoder Model ---")
print("An autoencoder learns to reconstruct normal data. Anomalies result in high reconstruction error.")

def build_autoencoder(input_shape):
    encoder_inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    # Latent space representation
    latent = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    
    encoder = keras.Model(encoder_inputs, latent, name="encoder")

    decoder_inputs = keras.Input(shape=latent.shape[1:]) # Input to decoder is the latent space shape
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(decoder_inputs)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoder_outputs = layers.Conv2DTranspose(input_shape[-1], (3, 3), activation='sigmoid', padding='same')(x)
    
    decoder = keras.Model(decoder_inputs, decoder_outputs, name="decoder")

    autoencoder_outputs = decoder(encoder(encoder_inputs))
    autoencoder = keras.Model(encoder_inputs, autoencoder_outputs, name="autoencoder")
    return autoencoder

if train_ds:
    autoencoder_model = build_autoencoder(IMAGE_SIZE + (3,))
    autoencoder_model.compile(optimizer='adam', loss='mse') # Mean Squared Error for reconstruction loss

    print("\nAutoencoder Model Summary:")
    autoencoder_model.summary()

    print("\nTraining Autoencoder on 'Normal' Data (a small subset for quick test)...")
    history = autoencoder_model.fit(
        train_ds,
        epochs=5, # Keep epochs low for quick notebook demo
        validation_data=val_ds,
        verbose=1
    )

    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Autoencoder Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.show()

    # --- Visualize Reconstruction ---
    print("\nVisualizing Original vs. Reconstructed Images:")
    sample_batch = next(iter(val_ds)) # Get a batch of validation images
    original_images = sample_batch[:min(5, BATCH_SIZE)] # Take first few images
    reconstructed_images = autoencoder_model.predict(original_images)

    n = len(original_images)
    plt.figure(figsize=(2 * n, 4))
    for i in range(n):
        # Original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(original_images[i])
        plt.title("Original")
        plt.axis("off")

        # Reconstructed
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed_images[i])
        plt.title("Reconstructed")
        plt.axis("off")
    plt.show()

    # --- Anomaly Scoring using Reconstruction Error ---
    print("\nCalculating Reconstruction Error for Normal vs. Anomaly Images:")
    
    # Calculate MSE for normal images
    normal_reconstruction_errors = []
    for batch in val_ds:
        reconstructed_batch = autoencoder_model.predict(batch)
        mse = tf.reduce_mean(tf.square(batch - reconstructed_batch), axis=(1, 2, 3))
        normal_reconstruction_errors.extend(mse.numpy())

    # Calculate MSE for anomaly images
    anomaly_reconstruction_errors = []
    if anomaly_ds:
        for batch in anomaly_ds:
            reconstructed_batch = autoencoder_model.predict(batch)
            mse = tf.reduce_mean(tf.square(batch - reconstructed_batch), axis=(1, 2, 3))
            anomaly_reconstruction_errors.extend(mse.numpy())

    plt.figure(figsize=(10, 6))
    sns.histplot(normal_reconstruction_errors, bins=50, kde=True, color='blue', label='Normal')
    if anomaly_ds and anomaly_reconstruction_errors:
        sns.histplot(anomaly_reconstruction_errors, bins=50, kde=True, color='red', label='Anomaly')
    plt.title('Distribution of Reconstruction Errors')
    plt.xlabel('Reconstruction Error (MSE)')
    plt.ylabel('Count')
    plt.legend()
    plt.show()

    print("Expected: Anomaly images should have higher reconstruction errors than normal images.")

else:
    print("Skipping Autoencoder exploration as data was not loaded.")

#### Gemini Version Integration:

In [ ]:
# Gemini Vision Integration for Anomaly Detection

print("--- Gemini Vision Integration ---")
print("Leveraging Gemini Vision for image understanding to aid anomaly detection.")

# --- Helper function to load image as GImage for Gemini ---
def load_gimage_from_gcs(gcs_uri):
    """Loads an image from GCS and converts to vertexai.preview.generative_models.Image."""
    try:
        bucket_name = gcs_uri.split('gs://')[1].split('/')[0]
        blob_name = '/'.join(gcs_uri.split('gs://')[1].split('/')[1:])
        bucket = gcs_client.get_bucket(bucket_name)
        blob = bucket.blob(blob_name)
        img_bytes = blob.download_as_bytes()
        return GImage.from_bytes(img_bytes)
    except Exception as e:
        print(f"Error loading image {gcs_uri} for Gemini: {e}")
        return None

# Get a few sample image URIs (mix of normal and anomaly if possible)
sample_image_uris = []
if normal_image_uris:
    sample_image_uris.extend(random.sample(normal_image_uris, min(3, len(normal_image_uris))))
if anomaly_image_uris:
    sample_image_uris.extend(random.sample(anomaly_image_uris, min(2, len(anomaly_image_uris))))

if sample_image_uris:
    print(f"\nAnalyzing {len(sample_image_uris)} sample images using Gemini Vision:")
    for i, uri in enumerate(sample_image_uris):
        print(f"\n--- Processing Image {i+1}: {uri.split('/')[-1]} ---")
        g_image = load_gimage_from_gcs(uri)

        if g_image:
            # Display image in notebook
            img_bytes = gcs_client.get_bucket(uri.split('gs://')[1].split('/')[0]).blob('/'.join(uri.split('gs://')[1].split('/')[1:])).download_as_bytes()
            display(Image.open(io.BytesIO(img_bytes)).resize((200, 200))) # display original image

            # --- 1. Generate Image Descriptions / Captions ---
            try:
                prompt_description = "Describe this image in detail, focusing on any objects, patterns, or unusual features."
                response_description = gemini_model_pro_vision.generate_content([g_image, prompt_description])
                print(f"Gemini Description:\n{response_description.text}")
            except Exception as e:
                print(f"Error generating description for {uri}: {e}")

            # --- 2. Anomaly-Specific Questioning ---
            # Ask targeted questions to identify anomalies based on expected 'normal' features
            prompt_anomaly_q = "Does this image show any signs of damage, wear, or unusual patterns for a machine part? Explain why or why not."
            try:
                response_anomaly_q = gemini_model_pro_vision.generate_content([g_image, prompt_anomaly_q])
                print(f"\nGemini Anomaly Analysis:\n{response_anomaly_q.text}")
            except Exception as e:
                print(f"Error with anomaly analysis for {uri}: {e}")

            # --- 3. (Conceptual) Generating Embeddings for Classification ---
            # As of current Gemini API, direct programmatic access to raw embeddings
            # for images/multimodal content isn't directly exposed for easy extraction
            # like text embeddings via `textembedding-gecko`. However, the model uses them
            # internally. You would typically use the generated text descriptions
            # and then get text embeddings from `textembedding-gecko` for downstream tasks,
            # or use the image itself directly in a multimodal classification model.
            
            # If `textembedding-gecko` is what you'd use for text features from Gemini's output:
            try:
                text_embeddings_model = aiplatform.TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
                # Use a part of Gemini's description to get an embedding
                if response_description and response_description.text:
                    description_embedding = text_embeddings_model.predict([response_description.text])
                    print(f"\nExample Text Embedding (from description): Shape {description_embedding.embeddings[0].values.shape}")
                    # These embeddings can be used as features for a separate anomaly classifier
            except Exception as e:
                print(f"Error getting text embedding (ensure textembedding-gecko is enabled): {e}")

            print("-" * 50)
        else:
            print(f"Skipping Gemini analysis for {uri} due to image loading error.")

    print("\nGemini Vision can be powerful for zero-shot anomaly detection by describing anomalies or for generating features (textual descriptions) to train other models.")
else:
    print("Skipping Gemini Vision integration as no sample images were available.")

#### Hyperparameter Tuning:

In [ ]:
# Hyperparameter Tuning with Vertex AI Vizier

print("--- Hyperparameter Tuning with Vertex AI Vizier ---")
print("Vertex AI Vizier systematically tunes hyperparameters to find optimal model performance.")

# --- 1. Define the Training Script ---
# This script will be executed by the Vizier study.
# It should accept hyperparameters as command-line arguments and report the objective metric.

TRAINING_SCRIPT_NAME = 'anomaly_trainer.py'
TRAINING_SCRIPT_CONTENT = f"""
import os
import argparse
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, optimizers
from google.cloud import storage
from sklearn.metrics import roc_auc_score, mean_squared_error

# Helper function to load image from GCS and preprocess
def load_and_preprocess_image(gcs_uri, image_size=({IMAGE_SIZE[0]}, {IMAGE_SIZE[1]})):
    img_bytes = tf.io.read_file(gcs_uri)
    img = tf.image.decode_jpeg(img_bytes, channels=3)
    img = tf.image.resize(img, image_size)
    img = img / 255.0
    return img

# Autoencoder Model Definition
def build_autoencoder(input_shape, latent_dim):
    encoder_inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_inputs)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    # Adjust latent dimension using Conv2D or Dense layers
    x = layers.Conv2D(latent_dim, (3, 3), activation='relu', padding='same')(x) # Latent layer

    decoder_inputs_shape = x.shape[1:] # Capture shape after latent layer
    
    x = layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoder_outputs = layers.Conv2DTranspose(input_shape[-1], (3, 3), activation='sigmoid', padding='same')(x)
    
    autoencoder = keras.Model(encoder_inputs, decoder_outputs, name="autoencoder")
    return autoencoder

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--learning_rate', type=float, default=0.001)
    parser.add_argument('--latent_dimension', type=int, default=128)
    parser.add_argument('--epochs', type=int, default=10) # Vizier will control this
    parser.add_argument('--batch_size', type=int, default=32)
    parser.add_argument('--normal_data_uri', type=str, required=True)
    parser.add_argument('--anomaly_data_uri', type=str, required=True)
    parser.add_argument('--model_dir', type=str, default=os.getenv('AIP_MODEL_DIR')) # Vertex AI managed output dir
    args = parser.parse_args()

    print(f"Starting training with LR={args.learning_rate}, LatentDim={args.latent_dimension}, Epochs={args.epochs}")

    # Load data from GCS
    gcs_client = storage.Client()
    normal_uris = [f"gs://{{args.normal_data_uri.split('gs://')[1]}}{blob.name}" for blob in gcs_client.list_blobs(args.normal_data_uri.split('gs://')[1].split('/')[0], prefix='/'.join(args.normal_data_uri.split('gs://')[1].split('/')[1:])) if blob.name.lower().endswith(('.jpg', '.jpeg', '.png'))]
    anomaly_uris = [f"gs://{{args.anomaly_data_uri.split('gs://')[1]}}{blob.name}" for blob in gcs_client.list_blobs(args.anomaly_data_uri.split('gs://')[1].split('/')[0], prefix='/'.join(args.anomaly_data_uri.split('gs://')[1].split('/')[1:])) if blob.name.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # For simplicity, use first X normal images for training/validation
    num_train_samples = min(2000, len(normal_uris)) # Use more data for actual training
    train_uris, val_uris = train_test_split(normal_uris[:num_train_samples], test_size=0.2, random_state=42)

    train_ds = tf.data.Dataset.from_tensor_slices(train_uris).map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(args.batch_size).cache().prefetch(tf.data.AUTOTUNE)
    val_ds = tf.data.Dataset.from_tensor_slices(val_uris).map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(args.batch_size).cache().prefetch(tf.data.AUTOTUNE)
    
    # For evaluation, load anomaly data and some normal data
    test_normal_uris = normal_uris[num_train_samples:min(num_train_samples + 500, len(normal_uris))]
    test_anomaly_uris = anomaly_uris[:min(500, len(anomaly_uris))] # Use up to 500 anomalies for test

    test_normal_ds = tf.data.Dataset.from_tensor_slices(test_normal_uris).map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(args.batch_size).cache().prefetch(tf.data.AUTOTUNE)
    test_anomaly_ds = tf.data.Dataset.from_tensor_slices(test_anomaly_uris).map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(args.batch_size).cache().prefetch(tf.data.AUTOTUNE)

    autoencoder_model = build_autoencoder(IMAGE_SIZE + (3,), args.latent_dimension)
    autoencoder_model.compile(optimizer=optimizers.Adam(learning_rate=args.learning_rate), loss='mse')

    history = autoencoder_model.fit(
        train_ds,
        epochs=args.epochs,
        validation_data=val_ds,
        verbose=0 # Run silently for Vizier
    )

    # --- Evaluate model and report metric to Vizier ---
    # Combine test datasets and get reconstruction errors
    all_test_images = []
    all_test_labels = [] # 0 for normal, 1 for anomaly

    for batch in test_normal_ds:
        all_test_images.append(batch)
        all_test_labels.extend([0] * len(batch))
    for batch in test_anomaly_ds:
        all_test_images.append(batch)
        all_test_labels.extend([1] * len(batch))
    
    if len(all_test_images) == 0:
        print("No test data available for evaluation. Skipping metric reporting.")
        exit() # Exit if no data to prevent errors

    all_test_images_tensor = tf.concat(all_test_images, axis=0)
    reconstructed_test_images = autoencoder_model.predict(all_test_images_tensor)
    reconstruction_errors = tf.reduce_mean(tf.square(all_test_images_tensor - reconstructed_test_images), axis=(1, 2, 3)).numpy()

    # Calculate AUC for anomaly detection (higher is better)
    # Labels: 0 (normal), 1 (anomaly)
    # Scores: reconstruction_errors (higher error = more anomalous)
    if len(np.unique(all_test_labels)) > 1: # Ensure both classes are present for AUC
        auc_score = roc_auc_score(all_test_labels, reconstruction_errors)
        print(f"Validation AUC: {{auc_score:.4f}}")
        # Report the metric to Vizier
        # Important: The metric name must match the one defined in the Vizier study config
        tf.summary.scalar('val_auc', auc_score, step=args.epochs) # For TensorBoard
        # Vertex AI Vizier automatically captures logs for `HP_METRIC_TAG` output or `tf.summary`
        # Or you can explicitly print as: print(f'hp_metric: {auc_score}')
    else:
        print("Not enough classes in test data to calculate AUC. Skipping metric reporting.")
    
    # Save the best model (optional, for later deployment)
    # autoencoder_model.save(args.model_dir) # Vertex AI handles this for experiments.
"""

# Write the training script to a file
with open(TRAINING_SCRIPT_NAME, 'w') as f:
    f.write(TRAINING_SCRIPT_CONTENT)

# --- 2. Upload the Training Script to GCS ---
TRAINING_PACKAGE_GCS_URI = f'gs://{GCS_BUCKET_NAME}/training_packages/{TRAINING_SCRIPT_NAME}'
blob = gcs_client.get_bucket(GCS_BUCKET_NAME).blob(f'training_packages/{TRAINING_SCRIPT_NAME}')
blob.upload_from_filename(TRAINING_SCRIPT_NAME)
print(f"\nTraining script uploaded to: {TRAINING_PACKAGE_GCS_URI}")

# --- 3. Define the Vizier Study Configuration ---
DISPLAY_NAME = f'anomaly-detection-vizier-study-{pd.Timestamp.now().strftime("%Y%m%d%H%M")}'
TRIAL_COUNT = 10 # Number of hyperparameter trials to run
PARALLEL_TRIAL_COUNT = 3 # Number of trials to run in parallel

# Define the hyperparameter search space
parameter_spec = [
    {"parameter_id": "learning_rate", "double_value_spec": {"min_value": 1e-4, "max_value": 1e-2}, "scale_type": "UNIT_LINEAR_SCALE"},
    {"parameter_id": "latent_dimension", "integer_value_spec": {"min_value": 64, "max_value": 256}, "scale_type": "UNIT_LINEAR_SCALE"},
    {"parameter_id": "epochs", "integer_value_spec": {"min_value": 5, "max_value": 20}, "scale_type": "UNIT_LINEAR_SCALE"}
]

# Define the objective metric (must match what your training script reports)
metric_spec = [{"metric_id": "val_auc", "goal": "MAXIMIZE"}]

# --- 4. Create and Run the Hyperparameter Tuning Job ---
if train_ds:
    try:
        hp_job = aiplatform.CustomContainerTrainingJob(
            display_name=DISPLAY_NAME,
            container_uri='gcr.io/deeplearning-platform-release/tf2-gpu.2-13', # Use a suitable TensorFlow container
            # Alternatively, if your custom image has everything, use it:
            # container_uri=f'us-central1-docker.pkg.dev/{PROJECT_ID}/your-repo/vertex-ai-research-env:latest',
            
            # The script and command to run it
            command=['python', TRAINING_SCRIPT_NAME],
            
            # Machine configuration for each trial
            machine_type='n1-standard-4', # Or more powerful for larger models/data
            accelerator_type='NVIDIA_TESLA_V100', # Use GPU
            accelerator_count=1,
            
            # The GCS paths for the training data
            args=[
                f'--normal_data_uri=gs://{GCS_BUCKET_NAME}/{GCS_NORMAL_DATA_PREFIX}',
                f'--anomaly_data_uri=gs://{GCS_BUCKET_NAME}/{GCS_ANOMALY_DATA_PREFIX}'
            ]
        )

        print("\nStarting Hyperparameter Tuning Job (Vizier Study)... This will take time.")
        hpt_job = hp_job.run_tuning(
            service_account=None, # Uses default compute service account
            parameter_spec=parameter_spec,
            metric_spec=metric_spec,
            max_trial_count=TRIAL_COUNT,
            parallel_trial_count=PARALLEL_TRIAL_COUNT
        )

        print(f"Hyperparameter Tuning Job '{hpt_job.display_name}' started.")
        print(f"Monitor job in console: https://console.cloud.google.com/vertex-ai/hyperparameter-tuning/jobs/{hpt_job.name.split('/')[-1]}?project={PROJECT_ID}&region={REGION}")

        # Wait for the job to complete and print results
        # hpt_job.wait_for_resource_creation() # Wait for job to be created
        # hpt_job.wait() # Wait for job to complete (can be long)
        # print("\nHyperparameter Tuning Job completed.")
        # print(f"Best Trial: {hpt_job.trials[0].parameters} with {hpt_job.trials[0].final_measurement.metrics[0].value}")
        print("\n(Note: The cell will return before the Vizier job completes. Check console for results.)")

    except Exception as e:
        print(f"Error starting Hyperparameter Tuning Job: {e}")
        print("Ensure your service account has 'Vertex AI User' role and necessary APIs are enabled.")
else:
    print("Skipping Hyperparameter Tuning as data was not loaded.")

#### Custom Training Logic:

In [ ]:
# Custom Training Logic and Loss Functions

print("--- Custom Training Logic and Loss Functions ---")
print("Implementing a custom training loop for anomaly detection with specialized loss.")

if train_ds and anomaly_ds:
    # --- 1. Define a Custom Autoencoder Model ---
    # We'll use a slightly different autoencoder or a more complex one
    def build_custom_autoencoder(input_shape, filter_base=32):
        encoder_inputs = keras.Input(shape=input_shape)
        x = layers.Conv2D(filter_base, (3, 3), activation='relu', padding='same')(encoder_inputs)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)
        x = layers.Conv2D(filter_base * 2, (3, 3), activation='relu', padding='same')(x)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)
        x = layers.Conv2D(filter_base * 4, (3, 3), activation='relu', padding='same')(x)
        latent = layers.MaxPooling2D((2, 2), padding='same')(x) # Example latent representation

        decoder_inputs = keras.Input(shape=latent.shape[1:])
        x = layers.Conv2DTranspose(filter_base * 4, (3, 3), activation='relu', padding='same')(decoder_inputs)
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Conv2DTranspose(filter_base * 2, (3, 3), activation='relu', padding='same')(x)
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Conv2DTranspose(filter_base, (3, 3), activation='relu', padding='same')(x)
        decoder_outputs = layers.UpSampling2D((2, 2))(x)
        decoder_outputs = layers.Conv2D(input_shape[-1], (3, 3), activation='sigmoid', padding='same')(decoder_outputs)
        
        autoencoder = keras.Model(encoder_inputs, decoder_outputs, name="custom_autoencoder")
        return autoencoder

    custom_autoencoder = build_custom_autoencoder(IMAGE_SIZE + (3,))

    # --- 2. Define a Custom Loss Function (e.g., Weighted MSE) ---
    # In anomaly detection, you might want to penalize reconstruction errors differently
    # or add a regularization term based on latent space properties.
    def weighted_reconstruction_loss(y_true, y_pred, weight_factor=1.5):
        mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=[1, 2, 3])
        # Example: if you had labels (0=normal, 1=anomaly), you could weight anomaly loss more
        # Here, we just use a constant weight factor for demonstration.
        return tf.reduce_mean(mse * weight_factor) # Simple weighting for demonstration

    # You can also use `losses.MeanSquaredError()` if no custom weighting is needed.
    # We'll use a standard MSE for the custom loop to keep focus on the loop itself.
    reconstruction_loss_fn = tf.keras.losses.MeanSquaredError()

    # --- 3. Implement a Custom Training Loop ---
    optimizer = optimizers.Adam(learning_rate=0.001)

    @tf.function
    def train_step(x_batch_normal):
        with tf.GradientTape() as tape:
            reconstructed_images = custom_autoencoder(x_batch_normal, training=True)
            loss = reconstruction_loss_fn(x_batch_normal, reconstructed_images)
        gradients = tape.gradient(loss, custom_autoencoder.trainable_variables)
        optimizer.apply_gradients(zip(gradients, custom_autoencoder.trainable_variables))
        return loss

    @tf.function
    def val_step(x_batch_normal):
        reconstructed_images = custom_autoencoder(x_batch_normal, training=False)
        loss = reconstruction_loss_fn(x_batch_normal, reconstructed_images)
        return loss

    EPOCHS = 10 # More epochs for a better demonstration
    train_losses = []
    val_losses = []

    print("\nStarting Custom Training Loop...")
    for epoch in range(EPOCHS):
        epoch_train_loss = 0.0
        num_train_batches = 0
        for batch_idx, x_batch_normal in enumerate(train_ds):
            loss = train_step(x_batch_normal)
            epoch_train_loss += loss
            num_train_batches += 1
        avg_train_loss = epoch_train_loss / num_train_batches
        train_losses.append(avg_train_loss.numpy())

        epoch_val_loss = 0.0
        num_val_batches = 0
        for batch_idx, x_batch_normal in enumerate(val_ds):
            loss = val_step(x_batch_normal)
            epoch_val_loss += loss
            num_val_batches += 1
        avg_val_loss = epoch_val_loss / num_val_batches
        val_losses.append(avg_val_loss.numpy())

        print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

    # Plot custom training losses
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Custom Train Loss')
    plt.plot(val_losses, label='Custom Val Loss')
    plt.title('Custom Autoencoder Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.show()

    print("\nCustom training loop complete.")

    # --- 4. Evaluate with Custom Metrics (AUC for Anomaly Detection) ---
    print("\nEvaluating Custom Autoencoder for Anomaly Detection (AUC)...")
    
    all_test_uris = []
    all_test_labels = [] # 0 for normal, 1 for anomaly

    # Load more normal samples for testing
    num_test_normal_samples = min(200, len(normal_image_uris) - len(train_uris) - len(val_uris))
    test_normal_uris = normal_image_uris[len(train_uris) + len(val_uris):len(train_uris) + len(val_uris) + num_test_normal_samples]

    all_test_uris.extend(test_normal_uris)
    all_test_labels.extend([0] * len(test_normal_uris))

    # Add anomaly samples
    num_test_anomaly_samples = min(200, len(anomaly_image_uris))
    test_anomaly_uris_subset = random.sample(anomaly_image_uris, num_test_anomaly_samples) # Random subset
    
    all_test_uris.extend(test_anomaly_uris_subset)
    all_test_labels.extend([1] * len(test_anomaly_uris_subset))

    if not all_test_uris:
        print("No test URIs available. Skipping AUC evaluation.")
    else:
        test_ds_combined = tf.data.Dataset.from_tensor_slices(all_test_uris).map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

        reconstruction_errors_test = []
        for batch in test_ds_combined:
            reconstructed_batch = custom_autoencoder(batch, training=False)
            mse = tf.reduce_mean(tf.square(batch - reconstructed_batch), axis=(1, 2, 3))
            reconstruction_errors_test.extend(mse.numpy())

        if len(np.unique(all_test_labels)) > 1:
            auc_score = roc_auc_score(all_test_labels, reconstruction_errors_test)
            precision, recall, _ = precision_recall_curve(all_test_labels, reconstruction_errors_test)
            pr_auc = auc(recall, precision)

            print(f"Anomaly Detection AUC Score: {auc_score:.4f}")
            print(f"Precision-Recall AUC Score: {pr_auc:.4f}")

            # Plot Precision-Recall Curve
            plt.figure(figsize=(8, 6))
            plt.plot(recall, precision, label=f'Precision-Recall curve (AUC = {pr_auc:.2f})')
            plt.xlabel('Recall')
            plt.ylabel('Precision')
            plt.title('Precision-Recall Curve for Anomaly Detection')
            plt.legend()
            plt.grid(True)
            plt.show()
        else:
            print("Not enough unique classes (normal/anomaly) in test set to calculate AUC/PR-AUC.")

else:
    print("Skipping Custom Training Logic as data was not loaded.")

#### Experiment Tracking:

In [ ]:
# Experiment Tracking with Vertex AI Experiments

print("--- Experiment Tracking with Vertex AI Experiments ---")
print("Logging model performance, parameters, and artifacts for reproducibility.")

if custom_autoencoder: # Assuming custom_autoencoder from previous cell
    EXPERIMENT_NAME = "Anomaly_Detection_Research"
    RUN_NAME = f"Autoencoder_Run_{pd.Timestamp.now().strftime('%Y%m%d%H%M%S')}"

    print(f"\nStarting Vertex AI Experiment '{EXPERIMENT_NAME}' with run '{RUN_NAME}'...")

    # Start an experiment run
    aiplatform.start_run(experiment=EXPERIMENT_NAME, run=RUN_NAME)

    try:
        # --- 1. Log Parameters ---
        aiplatform.log_params({
            "model_architecture": "CustomAutoencoder",
            "image_size": f"{IMAGE_SIZE[0]}x{IMAGE_SIZE[1]}",
            "epochs": EPOCHS, # From custom training loop
            "learning_rate": optimizer.learning_rate.numpy(),
            "loss_function": "MeanSquaredError",
            "batch_size": BATCH_SIZE
        })
        print("Logged experiment parameters.")

        # --- 2. Log Metrics (from previous custom training loop) ---
        # Log final metrics
        if 'auc_score' in locals() and 'pr_auc' in locals():
            aiplatform.log_metrics({
                "final_auc": auc_score,
                "final_pr_auc": pr_auc
            })
            print(f"Logged final metrics: AUC={auc_score:.4f}, PR_AUC={pr_auc:.4f}")
        else:
            print("No final AUC/PR_AUC to log (might be due to insufficient test data).")

        # Log training curve metrics per epoch
        if train_losses and val_losses:
            for i, (train_l, val_l) in enumerate(zip(train_losses, val_losses)):
                aiplatform.log_metrics(
                    metrics={
                        "train_loss": train_l,
                        "val_loss": val_l
                    },
                    step=i + 1
                )
            print(f"Logged per-epoch training and validation loss for {len(train_losses)} epochs.")
        else:
            print("No training curve losses to log.")

        # --- 3. Save and Log Model Checkpoints (Optional) ---
        # Save the trained model to GCS
        MODEL_GCS_PATH = f'gs://{GCS_BUCKET_NAME}/models/{RUN_NAME}/'
        custom_autoencoder.save(MODEL_GCS_PATH)
        print(f"\nModel saved to GCS: {MODEL_GCS_PATH}")

        # Log the saved model as an artifact in Vertex AI Experiments
        aiplatform.log_artifacts(
            artifacts=[
                aiplatform.Artifact.create(
                    schema_title='google.VertexModel', # Or a custom schema for your saved model
                    uri=MODEL_GCS_PATH,
                    display_name='Anomaly_Detection_Autoencoder',
                    metadata={'model_type': 'autoencoder', 'framework': 'tensorflow', 'gcs_path': MODEL_GCS_PATH}
                )
            ]
        )
        print("Logged model artifact.")

        # --- 4. End the Run ---
        aiplatform.end_run()
        print(f"\nExperiment run '{RUN_NAME}' ended.")
        print(f"View this run in Vertex AI Experiments: https://console.cloud.google.com/vertex-ai/experiments/experiments/{EXPERIMENT_NAME}/runs/{RUN_NAME}/details/metrics?project={PROJECT_ID}&region={REGION}")

    except Exception as e:
        print(f"Error during experiment tracking: {e}")
        aiplatform.end_run(status="ERROR") # End with error status if something went wrong
        print("Experiment run ended with an error status.")

else:
    print("Skipping Experiment Tracking as no custom autoencoder model was trained.")

print("\nVertex AI Experiments provides a centralized place to manage, compare, and analyze your ML experiments.")

### **Model Evaluation & Analysis:**

- **Quantitative Evaluation:** Develop Python scripts in notebooks to compute relevant metrics (e.g., F1-score, precision, recall, AUC, IoU for segmented anomalies) and compare against baselines.
- **Qualitative Analysis:** Visually inspect model predictions on anomalous and normal images, identify failure modes, and debug model behavior using visualization tools (e.g., Grad-CAM for saliency maps) within the notebook.
- **Adversarial Testing:** Explore how robust the anomaly detection model is to minor perturbations or adversarial examples.

### Loading and Exploring Datasets:

- Data Access: Connect to Cloud Storage buckets (e.g., for raw image data, processed features) and BigQuery tables.

In [ ]:
# Setup and Load Model/Data for Evaluation

import os
import io
import random
import json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses, models, optimizers

# Scikit-learn for evaluation metrics
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

# Google Cloud SDKs
from google.cloud import storage
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part, Image as GImage

# Set up GCP project and region (ensure these match your environment)
PROJECT_ID = os.getenv('GOOGLE_CLOUD_PROJECT')
REGION = 'us-central1' # Or your desired region

if not PROJECT_ID:
    try:
        _, project_id = aiplatform.initializer.global_config.get_client_options()
        PROJECT_ID = project_id
    except Exception:
        PROJECT_ID = "YOUR_GCP_PROJECT_ID" # REPLACE WITH YOUR PROJECT ID

print(f"Using Google Cloud Project: {PROJECT_ID}")
print(f"Using Google Cloud Region: {REGION}")

# Initialize Google Cloud Storage client
gcs_client = storage.Client(project=PROJECT_ID)

# --- Configuration for your dataset ---
GCS_NORMAL_DATA_PREFIX = 'raw_images/normal/' # REPLACE with your actual normal data path
GCS_ANOMALY_DATA_PREFIX = 'raw_images/anomaly/' # REPLACE with your actual anomaly data path
GCS_BUCKET_NAME = 'your-image-dataset-bucket' # REPLACE with your actual GCS bucket name

IMAGE_SIZE = (128, 128) # Standardize image size for models
BATCH_SIZE = 32

# --- Helper function to load and preprocess images for TensorFlow ---
def load_and_preprocess_image(gcs_uri, label=None, image_size=IMAGE_SIZE):
    img_bytes = tf.io.read_file(gcs_uri)
    img = tf.image.decode_jpeg(img_bytes, channels=3) # Adjust for PNG/BMP as needed
    img = tf.image.resize(img, image_size)
    img = img / 255.0 # Normalize to [0, 1]
    if label is not None:
        return img, label
    return img

# --- Load all normal and anomaly image URIs for testing ---
normal_image_uris = []
anomaly_image_uris = []
try:
    bucket = gcs_client.get_bucket(GCS_BUCKET_NAME)
    normal_blobs = list(bucket.list_blobs(prefix=GCS_NORMAL_DATA_PREFIX))
    normal_image_uris = [f"gs://{b.bucket.name}/{b.name}" for b in normal_blobs if b.name.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Found {len(normal_image_uris)} 'normal' images.")

    anomaly_blobs = list(bucket.list_blobs(prefix=GCS_ANOMALY_DATA_PREFIX))
    anomaly_image_uris = [f"gs://{b.bucket.name}/{b.name}" for b in anomaly_blobs if b.name.lower().endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Found {len(anomaly_image_uris)} 'anomaly' images.")

    # Create combined test dataset with labels
    # Use a reasonable number of samples for evaluation
    num_test_normal = min(500, len(normal_image_uris))
    num_test_anomaly = min(500, len(anomaly_image_uris))

    test_normal_uris = random.sample(normal_image_uris, num_test_normal)
    test_anomaly_uris = random.sample(anomaly_image_uris, num_test_anomaly)

    all_test_uris = test_normal_uris + test_anomaly_uris
    all_test_labels = [0] * len(test_normal_uris) + [1] * len(test_anomaly_uris) # 0 for normal, 1 for anomaly

    # Shuffle combined data to ensure randomness
    combined_list = list(zip(all_test_uris, all_test_labels))
    random.shuffle(combined_list)
    shuffled_test_uris, shuffled_test_labels = zip(*combined_list)

    test_ds_labeled = tf.data.Dataset.from_tensor_slices((list(shuffled_test_uris), list(shuffled_test_labels))).\
                                      map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE).\
                                      batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

    print(f"Prepared combined test dataset with {len(shuffled_test_uris)} samples.")

except Exception as e:
    print(f"Error loading initial image data: {e}")
    print("Ensure GCS_BUCKET_NAME, GCS_NORMAL_DATA_PREFIX, and GCS_ANOMALY_DATA_PREFIX are correct.")
    test_ds_labeled = None

# --- Load the trained model ---
# If you saved your model in the previous step, load it here.
# Otherwise, you might need to re-run the training cell or load a pre-trained model.
try:
    # Assuming the custom_autoencoder was defined and trained in Cell 5 of the previous section
    # Re-build the model structure to load weights if necessary
    def build_custom_autoencoder(input_shape, filter_base=32):
        encoder_inputs = keras.Input(shape=input_shape)
        x = layers.Conv2D(filter_base, (3, 3), activation='relu', padding='same')(encoder_inputs)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)
        x = layers.Conv2D(filter_base * 2, (3, 3), activation='relu', padding='same')(x)
        x = layers.MaxPooling2D((2, 2), padding='same')(x)
        x = layers.Conv2D(filter_base * 4, (3, 3), activation='relu', padding='same')(x)
        latent = layers.MaxPooling2D((2, 2), padding='same')(x) 

        decoder_inputs = keras.Input(shape=latent.shape[1:])
        x = layers.Conv2DTranspose(filter_base * 4, (3, 3), activation='relu', padding='same')(decoder_inputs)
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Conv2DTranspose(filter_base * 2, (3, 3), activation='relu', padding='same')(x)
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Conv2DTranspose(filter_base, (3, 3), activation='relu', padding='same')(x)
        decoder_outputs = layers.UpSampling2D((2, 2))(x)
        decoder_outputs = layers.Conv2D(input_shape[-1], (3, 3), activation='sigmoid', padding='same')(decoder_outputs)
        
        autoencoder = keras.Model(encoder_inputs, decoder_outputs, name="custom_autoencoder")
        return autoencoder

    model = build_custom_autoencoder(IMAGE_SIZE + (3,))
    # Attempt to load weights if a saved model path exists
    # If you saved your model to a specific GCS path, load from there:
    # SAVED_MODEL_GCS_PATH = 'gs://your-image-dataset-bucket/models/YourSpecificRunName/'
    # model = tf.keras.models.load_model(SAVED_MODEL_GCS_PATH)
    # For now, we'll assume a fresh model or one trained in the previous cell's execution.
    # If the custom_autoencoder object from the previous cell is still in memory, use it directly:
    if 'custom_autoencoder' in locals():
        model = custom_autoencoder
        print("Using the 'custom_autoencoder' object from the previous cell.")
    else:
        print("Warning: 'custom_autoencoder' not found. Model might not be loaded. Please ensure you ran previous training cells or load a saved model.")
        model = None # Set to None to prevent errors in subsequent cells
        
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

print("Setup and data/model loading complete.")

#### Quantitative Evaluation:

In [ ]:
# Quantitative Evaluation

print("--- Quantitative Evaluation ---")

if model and test_ds_labeled:
    # 1. Get predictions (reconstruction errors) and true labels
    true_labels = []
    reconstruction_errors = []
    original_images_for_viz = [] # Store a few for qualitative analysis

    print("Generating predictions and computing reconstruction errors...")
    for i, (images, labels) in enumerate(test_ds_labeled):
        reconstructed_images = model.predict(images)
        mse_batch = tf.reduce_mean(tf.square(images - reconstructed_images), axis=(1, 2, 3))
        
        reconstruction_errors.extend(mse_batch.numpy())
        true_labels.extend(labels.numpy())

        if i < 2: # Store a couple of batches for qualitative analysis later
            original_images_for_viz.append((images.numpy(), labels.numpy(), reconstructed_images.numpy(), mse_batch.numpy()))

    true_labels = np.array(true_labels)
    reconstruction_errors = np.array(reconstruction_errors)

    if len(np.unique(true_labels)) < 2:
        print("Not enough unique classes (normal/anomaly) in the test set to compute classification metrics. Please ensure your test set contains both types.")
    else:
        # 2. Calculate AUC-ROC
        # For anomaly detection, reconstruction error is the 'score', higher score = more anomalous
        auc_roc = roc_auc_score(true_labels, reconstruction_errors)
        print(f"\nArea Under the Receiver Operating Characteristic (AUC-ROC): {auc_roc:.4f}")

        # 3. Calculate Precision-Recall Curve and AUC-PR
        precision, recall, thresholds = precision_recall_curve(true_labels, reconstruction_errors)
        auc_pr = auc(recall, precision)
        print(f"Area Under the Precision-Recall Curve (AUC-PR): {auc_pr:.4f}")

        # Plot ROC curve
        fpr, tpr, _ = tf.keras.metrics.Roc(thresholds=np.linspace(0, 1, 100)).update_state(true_labels, reconstruction_errors).result()
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {auc_roc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve')
        plt.legend()
        plt.grid(True)

        # Plot Precision-Recall curve
        plt.subplot(1, 2, 2)
        plt.plot(recall, precision, label=f'Precision-Recall Curve (AUC = {auc_pr:.2f})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # 4. Determine optimal threshold and F1-score
        # A common approach is to find the threshold that maximizes F1-score
        f1_scores = []
        for thresh in thresholds:
            # Predictions: 1 if error > thresh, 0 otherwise
            y_pred = (reconstruction_errors >= thresh).astype(int)
            f1 = f1_score(true_labels, y_pred)
            f1_scores.append(f1)

        optimal_threshold_idx = np.argmax(f1_scores)
        optimal_threshold = thresholds[optimal_threshold_idx]
        optimal_f1_score = f1_scores[optimal_threshold_idx]

        print(f"\nOptimal Threshold (maximizing F1-score): {optimal_threshold:.4f}")
        print(f"F1-Score at Optimal Threshold: {optimal_f1_score:.4f}")

        # Confusion Matrix at optimal threshold
        y_pred_optimal = (reconstruction_errors >= optimal_threshold).astype(int)
        cm = confusion_matrix(true_labels, y_pred_optimal)
        print("\nConfusion Matrix at Optimal Threshold:")
        print(cm)
        print(f"  True Negatives (TN): {cm[0, 0]}") # Correctly identified normal
        print(f"  False Positives (FP): {cm[0, 1]}") # Normal incorrectly identified as anomaly
        print(f"  False Negatives (FN): {cm[1, 0]}") # Anomaly incorrectly identified as normal
        print(f"  True Positives (TP): {cm[1, 1]}") # Correctly identified anomaly

        # Optional: IoU for Segmented Anomalies
        # This metric is relevant if your anomaly detection model outputs a pixel-level mask
        # indicating the anomalous regions.
        # For a simple autoencoder, the "anomaly" is typically defined by high reconstruction error per pixel.
        # To calculate IoU, you would need:
        # 1. Ground truth anomaly masks (pixel-level annotations for anomalies).
        # 2. A method to convert your model's output (e.g., pixel-wise reconstruction error)
        #    into a binary segmentation mask (e.g., thresholding pixel errors).

        # Example Placeholder for IoU calculation (conceptual)
        # if your model was a U-Net that predicts an anomaly mask:
        # def calculate_iou(y_true_mask, y_pred_mask):
        #     intersection = np.logical_and(y_true_mask, y_pred_mask).sum()
        #     union = np.logical_or(y_true_mask, y_pred_mask).sum()
        #     if union == 0: return 1.0 # No pixels in either mask, perfect match
        #     return intersection / union

        # if anomaly_segmentation_available:
        #     iou_scores = []
        #     for original, true_mask, predicted_mask_logits in test_ds_segmentation:
        #         predicted_mask = (tf.nn.sigmoid(predicted_mask_logits) > 0.5).numpy().astype(int)
        #         iou_scores.append(calculate_iou(true_mask, predicted_mask))
        #     mean_iou = np.mean(iou_scores)
        #     print(f"\nMean Intersection over Union (IoU) for segmented anomalies: {mean_iou:.4f}")
        # else:
        print("\nIoU calculation is not applicable for a simple autoencoder anomaly detection setup (which outputs image-level reconstruction error). It applies to pixel-level anomaly segmentation.")

else:
    print("Skipping quantitative evaluation as model or test data is not available.")

#### Quantitative Analysis:

In [ ]:
# Qualitative Analysis

print("--- Qualitative Analysis ---")

if model and 'original_images_for_viz' in locals() and original_images_for_viz:
    print("\nVisualizing Original, Reconstructed, and Error Maps for Sample Images:")

    for batch_num, (originals, labels, reconstructions, errors) in enumerate(original_images_for_viz):
        print(f"\n--- Batch {batch_num + 1} ---")
        num_display = min(5, len(originals)) # Display up to 5 images from this batch

        plt.figure(figsize=(num_display * 3, 9))
        for i in range(num_display):
            is_anomaly = labels[i] == 1
            title_prefix = "Anomaly" if is_anomaly else "Normal"

            # Original Image
            ax = plt.subplot(3, num_display, i + 1)
            plt.imshow(originals[i])
            plt.title(f"{title_prefix}\nOriginal (Error: {errors[i]:.4f})")
            plt.axis("off")

            # Reconstructed Image
            ax = plt.subplot(3, num_display, i + 1 + num_display)
            plt.imshow(reconstructions[i])
            plt.title(f"{title_prefix}\nReconstructed")
            plt.axis("off")

            # Error Map (Absolute Difference)
            ax = plt.subplot(3, num_display, i + 1 + 2 * num_display)
            error_map = np.abs(originals[i] - reconstructions[i])
            plt.imshow(np.mean(error_map, axis=-1), cmap='hot') # Mean across channels
            plt.title(f"{title_prefix}\nError Map")
            plt.colorbar(ax=ax, fraction=0.046, pad=0.04) # Add colorbar
            plt.axis("off")
        plt.tight_layout()
        plt.show()

    print("\nObserve: Anomalies should generally have higher reconstruction errors and more 'hot' regions in their error maps.")
    print("Failure modes might include: normal images with high error (false positives), or anomalies with low error (false negatives).")


    # --- Grad-CAM for Saliency Maps ---
    # Grad-CAM helps understand which parts of the input image contributed most to the model's decision
    # (in this case, reconstruction error). For an autoencoder, we're interested in regions
    # that lead to high reconstruction loss.

    print("\n--- Grad-CAM for Saliency Maps on Autoencoder Errors ---")
    print("Note: Grad-CAM is typically applied to classification models by targeting a specific class logit.")
    print("For autoencoders, we can adapt it to highlight regions contributing to high reconstruction error.")

    def make_gradcam_heatmap(img_array, model, last_conv_layer_name="max_pooling2d_1", target_layer=None):
        """
        Generates a Grad-CAM heatmap for an autoencoder based on reconstruction error.
        
        Args:
            img_array (tf.Tensor): The input image tensor.
            model (tf.keras.Model): The autoencoder model.
            last_conv_layer_name (str): Name of the last convolutional layer in the encoder part.
                                        Adjust this based on your model's summary.
        Returns:
            np.ndarray: The heatmap.
        """
        if target_layer is None:
            # Try to infer the last conv layer if not provided
            try:
                for layer in reversed(model.layers):
                    if isinstance(layer, layers.Conv2D) or isinstance(layer, layers.Conv2DTranspose):
                        last_conv_layer_name = layer.name
                        print(f"Using inferred last convolutional layer: {last_conv_layer_name}")
                        break
                if not last_conv_layer_name:
                    raise ValueError("Could not find a convolutional layer in the model.")
            except Exception as e:
                print(f"Could not infer last conv layer: {e}. Please manually specify 'last_conv_layer_name'.")
                return None

        # Create a model that outputs the feature maps from the last conv layer
        # and the reconstructed output
        grad_model = tf.keras.models.Model(
            model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
        )

        with tf.GradientTape() as tape:
            # The input image is float32 and needs to be watched by the tape for gradients
            inputs = tf.cast(img_array, tf.float32)
            tape.watch(inputs)

            conv_outputs, predictions = grad_model(inputs)
            
            # The "loss" to compute gradients for is the reconstruction error
            # We want to know what features cause high error
            # For simplicity, we can take the mean reconstruction error over the image
            # Or, for more localized maps, sum errors per pixel
            loss = tf.reduce_mean(tf.square(inputs - predictions))
            # Or, if you want a more class-discriminative map, and if you had anomaly scores
            # loss = autoencoder_model.get_anomaly_score(inputs) # Requires a custom method for this.
            # For a simple autoencoder, loss is reconstruction error.

        # Get gradients of the loss with respect to the last conv layer's feature maps
        grads = tape.gradient(loss, conv_outputs)

        # Global average pooling of the gradients
        # This gives a "weight" for each feature map channel
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

        # Multiply each channel's feature map by its importance weight
        conv_outputs = conv_outputs[0] # Remove batch dimension
        heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap) # Remove last dimension of 1

        # ReLU on heatmap to only consider positive contributions
        heatmap = tf.maximum(heatmap, 0)
        
        # Normalize heatmap to [0, 1] for visualization
        max_val = tf.reduce_max(heatmap)
        if max_val == 0:
            return np.zeros_like(heatmap.numpy()) # Avoid division by zero
        heatmap /= max_val
        return heatmap.numpy()

    # Select a few images (normal and anomaly) for Grad-CAM
    grad_cam_sample_uris = []
    if normal_image_uris:
        grad_cam_sample_uris.extend(random.sample(normal_image_uris, min(2, len(normal_image_uris))))
    if anomaly_image_uris:
        grad_cam_sample_uris.extend(random.sample(anomaly_image_uris, min(2, len(anomaly_image_uris))))

    if grad_cam_sample_uris and model:
        for i, uri in enumerate(grad_cam_sample_uris):
            print(f"\nProcessing image for Grad-CAM: {uri.split('/')[-1]}")
            original_image = load_and_preprocess_image(uri)
            
            # Get the input layer's name dynamically if available or assume 'input_1'
            input_layer_name = model.input_names[0] if model.input_names else 'input_1'
            
            # Try to find a suitable last convolutional layer
            last_conv_layer_to_use = None
            for layer in reversed(model.layers):
                if isinstance(layer, layers.Conv2D) and 'encoder' in layer.name: # Focus on encoder part
                    last_conv_layer_to_use = layer.name
                    break
            if last_conv_layer_to_use is None:
                print("Could not find a suitable last convolutional layer in the encoder. Please check your model architecture.")
                continue

            heatmap = make_gradcam_heatmap(
                tf.expand_dims(original_image, axis=0),
                model,
                last_conv_layer_name=last_conv_layer_to_use # Adjust this based on your model summary
            )

            if heatmap is not None:
                # Resize heatmap to original image size
                heatmap = np.uint8(255 * heatmap)
                jet_colors = plt.cm.jet(np.arange(256))[:, :3]
                jet_heatmap = jet_colors[heatmap]
                jet_heatmap = keras.utils.array_to_img(jet_heatmap)
                jet_heatmap = jet_heatmap.resize((original_image.shape[1], original_image.shape[0]))
                jet_heatmap = keras.utils.img_to_array(jet_heatmap)

                # Superimpose the heatmap on the original image
                superimposed_img = jet_heatmap * 0.4 + original_image * 255.0 # Alpha blending
                superimposed_img = keras.utils.array_to_img(superimposed_img)

                plt.figure(figsize=(10, 5))
                plt.subplot(1, 3, 1)
                plt.imshow(original_image)
                plt.title("Original Image")
                plt.axis('off')

                plt.subplot(1, 3, 2)
                plt.imshow(Image.fromarray(heatmap)) # Raw heatmap
                plt.title("Grad-CAM Heatmap")
                plt.axis('off')

                plt.subplot(1, 3, 3)
                plt.imshow(superimposed_img)
                plt.title("Superimposed Heatmap")
                plt.axis('off')
                plt.tight_layout()
                plt.show()
            else:
                print(f"Skipping Grad-CAM for {uri} due to layer issue.")
    else:
        print("Skipping Grad-CAM as no sample images or model are available.")

else:
    print("Skipping qualitative analysis as model or test data for visualization is not available.")

#### Adverarial Testing:

In [ ]:
# Adversarial Testing

print("--- Adversarial Testing ---")
print("Exploring model robustness against minor perturbations/adversarial examples.")

if model and anomaly_image_uris:
    # Adversarial attacks aim to make an anomaly look normal, or vice versa.
    # For an autoencoder-based anomaly detector, an attack would try to reduce
    # the reconstruction error of an anomalous image.

    # --- 1. Select an example anomaly image ---
    sample_anomaly_uri = random.choice(anomaly_image_uris)
    original_anomaly_image = load_and_preprocess_image(sample_anomaly_uri)
    original_anomaly_image_tensor = tf.expand_dims(original_anomaly_image, axis=0) # Add batch dim

    # Calculate initial reconstruction error for the anomaly
    initial_reconstruction = model.predict(original_anomaly_image_tensor)
    initial_error = tf.reduce_mean(tf.square(original_anomaly_image_tensor - initial_reconstruction)).numpy()
    print(f"\nOriginal Anomaly: {sample_anomaly_uri.split('/')[-1]}")
    print(f"Initial Reconstruction Error: {initial_error:.6f}")

    # Display original anomaly image
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(original_anomaly_image.numpy())
    plt.title(f"Original Anomaly\nError: {initial_error:.4f}")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(initial_reconstruction[0])
    plt.title("Reconstructed Anomaly")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    error_map = np.abs(original_anomaly_image.numpy() - initial_reconstruction[0])
    plt.imshow(np.mean(error_map, axis=-1), cmap='hot')
    plt.title("Initial Error Map")
    plt.colorbar(ax=plt.gca(), fraction=0.046, pad=0.04)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    # --- 2. Implement a simple adversarial attack (FGSM - Fast Gradient Sign Method) ---
    # Goal: Find a small perturbation that reduces the reconstruction error of an anomaly.
    # This involves taking gradients of the error w.r.t. the input image.

    loss_object = tf.keras.losses.MeanSquaredError()

    def create_adversarial_example(model, input_image, epsilon):
        # input_image must be a tensor and differentiable
        input_image_tensor = tf.cast(input_image, tf.float32)
        with tf.GradientTape() as tape:
            tape.watch(input_image_tensor)
            reconstructed_image = model(input_image_tensor)
            # The loss here is what we want to minimize (reconstruction error)
            loss = loss_object(input_image_tensor, reconstructed_image)

        # Get the gradients of the loss with respect to the input image
        gradient = tape.gradient(loss, input_image_tensor)
        
        # Take the sign of the gradients to create the perturbation
        signed_grad = tf.sign(gradient)
        
        # Create the adversarial example by moving in the *negative* direction of the gradient
        # to reduce the loss. Epsilon controls the magnitude of the perturbation.
        adversarial_example = input_image_tensor - epsilon * signed_grad
        
        # Clip the perturbed image to be within valid pixel range [0, 1]
        adversarial_example = tf.clip_by_value(adversarial_example, 0, 1)
        
        return adversarial_example

    EPSILON = 0.05 # Magnitude of perturbation (adjust this!)

    print(f"\nAttempting to generate adversarial example with epsilon = {EPSILON}...")
    
    # Generate the adversarial example
    adversarial_anomaly_image_tensor = create_adversarial_example(
        model, 
        original_anomaly_image_tensor, 
        epsilon=EPSILON
    )

    # Calculate reconstruction error for the adversarial example
    adversarial_reconstruction = model.predict(adversarial_anomaly_image_tensor)
    adversarial_error = tf.reduce_mean(tf.square(adversarial_anomaly_image_tensor - adversarial_reconstruction)).numpy()

    print(f"Adversarial Reconstruction Error: {adversarial_error:.6f}")
    print(f"Error Change: {initial_error - adversarial_error:.6f}")

    # Display results
    plt.figure(figsize=(15, 5))

    plt.subplot(1, 4, 1)
    plt.imshow(original_anomaly_image.numpy())
    plt.title(f"Original Anomaly\nError: {initial_error:.4f}")
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.imshow(adversarial_anomaly_image_tensor[0].numpy())
    plt.title(f"Adversarial Anomaly\nError: {adversarial_error:.4f}")
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(initial_reconstruction[0].numpy())
    plt.title("Original Anomaly Rec.")
    plt.axis('off')

    plt.subplot(1, 4, 4)
    plt.imshow(adversarial_reconstruction[0].numpy())
    plt.title("Adversarial Anomaly Rec.")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # Visualize the perturbation itself
    perturbation = adversarial_anomaly_image_tensor[0].numpy() - original_anomaly_image.numpy()
    plt.figure(figsize=(5, 5))
    plt.imshow((perturbation + 1) / 2) # Normalize to [0,1] for display
    plt.title(f"Perturbation (Epsilon={EPSILON})")
    plt.axis('off')
    plt.show()

    print("\nObservation:")
    print("If the attack was successful, the 'Adversarial Anomaly' image should look very similar to the 'Original Anomaly',")
    print("but its reconstruction error should be significantly lower, potentially below your anomaly detection threshold.")
    print("This indicates a vulnerability where a slightly perturbed anomalous input could be misclassified as normal.")

else:
    print("Skipping Adversarial Testing as model or anomaly data is not available.")

print("\nModel evaluation and analysis complete.")

### **Containerization & Reproducibility for Model Training/Serving:**

- **Dockerfile Creation:** Write Dockerfiles to package their custom TensorFlow training code and model serving logic into reproducible containers (templates/custom_training_job.py, templates/custom_prediction_routine.py).
- **Local Docker Testing:** Build and test Docker images locally within the Workbench terminal before pushing to Artifact Registry.

#### Dockerfile Creation: 

Containerization is a fundamental MLOps practice that packages your machine learning code, dependencies, and environment into isolated, portable units called containers. This ensures that your model trains and serves consistently across different environments, from local development to production on Google Cloud Platform (GCP) with Vertex AI.

In these cells, we will:
1.  **Create Dockerfiles:** Define the build process for custom TensorFlow training and prediction routines.
2.  **Build Docker Images:** Transform your Dockerfiles into runnable images.
3.  **Test Locally:** Verify the container's functionality within the Vertex Workbench environment before pushing to Artifact Registry.

### Prepare Your Model Code and Dependencies

Before we create the Dockerfiles, ensure your TensorFlow training and prediction code is organized and ready. We'll assume you have two Python scripts:
- `templates/custom_training_job.py`: Contains your TensorFlow model training logic.
- `templates/custom_prediction_routine.py`: Contains your custom model serving logic, including how to load your trained model and make predictions.

**Directory Structure (Example):**

In [ ]:
import os

# Create dummy directories and files for demonstration if they don't exist
os.makedirs('templates', exist_ok=True)
os.makedirs('Dockerfiles', exist_ok=True)

# Dummy training script
training_code = """
import tensorflow as tf
import os

print("Running custom training job...")

# Simulate model training
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create a dummy model directory for saving
model_dir = os.environ.get('AIP_MODEL_DIR', 'model_output')
os.makedirs(model_dir, exist_ok=True)

# Simulate saving the model
model_save_path = os.path.join(model_dir, 'my_model')
tf.saved_model.save(model, model_save_path)
print(f"Dummy model saved to: {model_save_path}")

print("Custom training job finished.")
"""

with open('templates/custom_training_job.py', 'w') as f:
    f.write(training_code)

# Dummy prediction script
prediction_code = """
import tensorflow as tf
import numpy as np
import os
import json

class CustomPredictionRoutine(object):

    def __init__(self):
        self._model = None

    def load(self, model_path: str):
        print(f"Loading model from: {model_path}")
        self._model = tf.saved_model.load(model_path)
        print("Model loaded successfully.")

    def predict(self, instances):
        if self._model is None:
            raise RuntimeError("Model not loaded. Call load() first.")
        
        # Assume instances are already preprocessed numpy arrays
        predictions = self._model(tf.constant(instances, dtype=tf.float32)).numpy().tolist()
        return predictions

if __name__ == '__main__':
    # For local testing purposes
    # Set dummy AIP_MODEL_DIR to simulate Vertex AI environment
    os.environ['AIP_MODEL_DIR'] = 'model_output/my_model' 
    
    # Create dummy model artifacts for local testing
    if not os.path.exists('model_output/my_model'):
        print("Creating dummy model for local prediction routine testing...")
        dummy_model = tf.keras.Sequential([
            tf.keras.layers.Dense(10, activation='relu', input_shape=(784,)),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
        dummy_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        tf.saved_model.save(dummy_model, 'model_output/my_model')
        print("Dummy model created.")

    routine = CustomPredictionRoutine()
    routine.load(os.environ['AIP_MODEL_DIR'])
    
    # Example prediction
    dummy_input = np.random.rand(1, 784).tolist()
    predictions = routine.predict(dummy_input)
    print("Local prediction routine test completed. Sample prediction:")
    print(predictions[0][:5]) # Print first 5 prediction values
"""

with open('templates/custom_prediction_routine.py', 'w') as f:
    f.write(prediction_code)

print("Dummy `templates/custom_training_job.py` and `templates/custom_prediction_routine.py` created.")

## Dockerfile Creation

We will create two separate Dockerfiles: one for the training job and one for the prediction routine. This separation adheres to best practices for MLOps, allowing independent scaling and deployment of each component.

For base images, it's highly recommended to use Google Cloud's [Deep Learning Containers](https://cloud.google.com/deep-learning-containers/docs/choosing-container). These images are pre-installed with popular ML frameworks (TensorFlow, PyTorch) and optimized for GCP.

In [ ]:
# BASE VERSION
# Base-cu124 CUDA 12.4 (Python 3.10) CUDA 12.4 us-docker.pkg.dev/deeplearning-platform-release/gcr.io/base-cu124.py310

# TENSORFLOW VERSION
# 2.17 (Python 3.10) 2.17.0 CPU only us-docker.pkg.dev/deeplearning-platform-release/gcr.io/tf2-cpu.2-17.py310
# 2.17 (Python 3.10) 2.17.0 CUDA 12.3 us-docker.pkg.dev/deeplearning-platform-release/gcr.io/tf2-cu123.2-17.py310

# PYTORCH
# 2.4 (Python 3.10) 2.4.0 CUDA 12.4 us-docker.pkg.dev/deeplearning-platform-release/gcr.io/pytorch-cu124.2-4.py310

# TEXT GENERATION INFERENCE CONTAINERS 
# TGI 2.4 2.4.0 CUDA 12.4 us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-generation-inference-cu124.2-4.ubuntu2204.py311

# TEXT EMBEDDINGS INFERENCE CONTAINERS 
# TEI 1.5 1.5.1 CUDA 12.2 us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-embeddings-inference-cu122.1-5.ubuntu2204
# TEI 1.5 1.5.1 CPU us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-text-embeddings-inference-cpu.1-5

# PYTORCH INFERENCE CONTAINERS
# PyTorch 2.3 2.3.1 CUDA 12.1 4.46.1 us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-pytorch-inference-cu121.2-3.transformers.4-46.ubuntu2204.py311
# PyTorch 2.3 2.3.1 CPU 4.46.1 us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-pytorch-inference-cpu.2-3.transformers.4-46.ubuntu2204.py311

# PYTORCH TRAINING CONTAINERS
# PyTorch 2.3 2.3.0 CUDA 12.1 4.42.3 us-docker.pkg.dev/deeplearning-platform-release/gcr.io/huggingface-pytorch-training-cu121.2-3.transformers.4-42.ubuntu2204.py310

# vLLM INFERENCE CONTAINERS
# PyTorch 2.4 2.4.0 CUDA 12.1 us-docker.pkg.dev/deeplearning-platform-release/vertex-model-garden/vllm-inference.cu121.0-5.ubuntu2204.py310

In [ ]:
%%writefile Dockerfiles/Dockerfile.train

# Use a Deep Learning Container image with TensorFlow for training
FROM us-docker.pkg.dev/vertex-ai/training/tf-gpu.2-13.cpu:latest

# Set the working directory in the container
WORKDIR /app

# Install any additional Python dependencies for training
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy your training code into the container
COPY templates/custom_training_job.py /app/custom_training_job.py

# Set the entrypoint for the training job
# Vertex AI Training expects your entrypoint to run your training script
ENTRYPOINT ["python", "custom_training_job.py"]

In [ ]:
%%writefile Dockerfiles/Dockerfile.predict
# Use a Deep Learning Container image with TensorFlow for prediction
# A CPU image is often sufficient for serving, but you can choose GPU if needed.
FROM us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-13:latest

# Set the working directory in the container
WORKDIR /app

# Install any additional Python dependencies for serving
# (e.g., if custom_prediction_routine.py has specific libraries)
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy your custom prediction routine code into the container
COPY templates/custom_prediction_routine.py /app/custom_prediction_routine.py

# Vertex AI Custom Prediction expects an HTTP server for prediction.
# The base images for prediction typically come with the necessary server (e.g., FastAPI or Flask)
# and automatically discover your custom_prediction_routine.py if it implements the required methods (load, predict).
# Ensure your custom_prediction_routine.py adheres to the Vertex AI custom prediction routine interface.
# For TensorFlow, the tf2-cpu image handles this automatically.
# No explicit ENTRYPOINT needed if using the built-in Vertex AI prediction server logic.

In [ ]:
%%writefile requirements.txt
tensorflow~=2.13.0
numpy
# Add any other libraries your training or prediction code needs

#### Local Docker Testing:

Before pushing your Docker images to Artifact Registry and deploying them on Vertex AI, it's crucial to test them locally within your Vertex Workbench Jupyter Notebook environment. This helps catch issues early, saving time and compute costs.

You'll use the Workbench terminal for these commands.

**Steps:**
1.  **Build the Docker Image:** Use `docker build` to create your container image.
2.  **Run the Docker Container:** Use `docker run` to execute your container and test its functionality.

**Important Note:** The Vertex AI Workbench environment has Docker pre-installed and configured. You can directly run `docker` commands in the terminal.

In [ ]:
# Navigate to the directory containing your Dockerfiles
cd Dockerfiles

# Define an image name and tag
export TRAIN_IMAGE_NAME="my-tf-trainer"
export TRAIN_IMAGE_TAG="v1"

# Build the training Docker image
# The '.' indicates the build context, which is the current directory (Dockerfiles/)
# Adjust the build context if your templates directory is not relative to Dockerfiles
docker build -f Dockerfile.train -t ${TRAIN_IMAGE_NAME}:${TRAIN_IMAGE_TAG} ../.

# Go back to the notebook's root directory for next steps
cd ..

In [ ]:
# Run the training container locally
# You might want to mount a local volume to simulate model output or data input
# For this simple example, we'll just run it to see the print statements.
# In a real scenario, you'd want to pass arguments or mount data.
docker run --rm ${TRAIN_IMAGE_NAME}:${TRAIN_IMAGE_TAG}

# The --rm flag automatically removes the container after it exits.

In [ ]:
# Navigate to the directory containing your Dockerfiles
cd Dockerfiles

# Define an image name and tag
export PREDICT_IMAGE_NAME="my-tf-predictor"
export PREDICT_IMAGE_TAG="v1"

# Build the prediction Docker image
docker build -f Dockerfile.predict -t ${PREDICT_IMAGE_NAME}:${PREDICT_IMAGE_TAG} ../.

# Go back to the notebook's root directory for next steps
cd ..

In [ ]:
# For local testing of prediction routines, you typically need to:
# 1. Have a saved model available locally.
# 2. Simulate the input data.
# 3. Call the prediction endpoint/method.

# We set a dummy AIP_MODEL_DIR in custom_prediction_routine.py for local testing.
# When running in Vertex AI, this environment variable is automatically provided.

# Run the prediction container locally.
# The entrypoint for prediction images is often managed by Vertex AI's serving runtime.
# To test locally, you'd typically run the script directly or simulate the HTTP server.
# Our dummy custom_prediction_routine.py has a `if __name__ == '__main__'` block for this.
docker run --rm -v $(pwd)/model_output:/app/model_output ${PREDICT_IMAGE_NAME}:${PREDICT_IMAGE_TAG} python /app/custom_prediction_routine.py

# Explanation:
# - `docker run --rm`: Runs the container and removes it after exit.
# - `-v $(pwd)/model_output:/app/model_output`: Mounts your local 'model_output' directory
#   (where the dummy model was saved by the training script) into the container's `/app/model_output`
#   location. This makes the saved model available inside the container for prediction.
# - `${PREDICT_IMAGE_NAME}:${PREDICT_IMAGE_TAG}`: Specifies the image to run.
# - `python /app/custom_prediction_routine.py`: Overrides the default entrypoint (if any)
#   to directly execute your prediction routine script for local testing.
#   In a real Vertex AI deployment, the serving runtime would handle calling the `load` and `predict`
#   methods of your `CustomPredictionRoutine` class automatically.

In [ ]:
## Pushing to Artifact Registry

Once you've successfully tested your Docker images locally, the next step in the MLOps pipeline is to push them to Google Cloud Artifact Registry. This makes your images accessible for Vertex AI Training and Vertex AI Prediction.

**Commands you would run in the Workbench Terminal:**

```bash
# Set your GCP Project ID and region
export PROJECT_ID=$(gcloud config get-value project)
export REGION="us-central1" # Or your desired region

# Define your Artifact Registry repository (create if it doesn't exist)
export AR_REPO="vertex-ai-containers"
gcloud artifacts repositories create <span class="math-inline">\{AR\_REPO\} \-\-repository\-format\=docker \-\-location\=</span>{REGION} --description="Docker images for Vertex AI" --async --quiet

# Authenticate Docker to Artifact Registry
gcloud auth configure-docker ${REGION}-docker.pkg.dev

# Tag the local images with the Artifact Registry path
docker tag <span class="math-inline">\{TRAIN\_IMAGE\_NAME\}\:</span>{TRAIN_IMAGE_TAG} <span class="math-inline">\{REGION\}\-docker\.pkg\.dev/</span>{PROJECT_ID}/<span class="math-inline">\{AR\_REPO\}/</span>{TRAIN_IMAGE_NAME}:${TRAIN_IMAGE_TAG}
docker tag <span class="math-inline">\{PREDICT\_IMAGE\_NAME\}\:</span>{PREDICT_IMAGE_TAG} <span class="math-inline">\{REGION\}\-docker\.pkg\.dev/</span>{PROJECT_ID}/<span class="math-inline">\{AR\_REPO\}/</span>{PREDICT_IMAGE_NAME}:${PREDICT_IMAGE_TAG}

# Push the images to Artifact Registry
docker push <span class="math-inline">\{REGION\}\-docker\.pkg\.dev/</span>{PROJECT_ID}/<span class="math-inline">\{AR\_REPO\}/</span>{TRAIN_IMAGE_NAME}:${TRAIN_IMAGE_TAG}
docker push <span class="math-inline">\{REGION\}\-docker\.pkg\.dev/</span>{PROJECT_ID}/<span class="math-inline">\{AR\_REPO\}/</span>{PREDICT_IMAGE_NAME}:<span class="math-inline">\{PREDICT\_IMAGE\_TAG\}
echo "Images pushed to Artifact Registry\:"
echo "</span>{REGION}-docker.pkg.dev/<span class="math-inline">\{PROJECT\_ID\}/</span>{AR_REPO}/<span class="math-inline">\{TRAIN\_IMAGE\_NAME\}\:</span>{TRAIN_IMAGE_TAG}"
echo "<span class="math-inline">\{REGION\}\-docker\.pkg\.dev/</span>{PROJECT_ID}/<span class="math-inline">\{AR\_REPO\}/</span>{PREDICT_IMAGE_NAME}:${PREDICT_IMAGE_TAG}"

**NOTE:** This structured notebook will provide a clear and actionable guide for AI researchers, enabling them to leverage containerization effectively for their MLOps workflows on GCP. Remember to emphasize the importance of local testing to quickly iterate and debug their container images before deploying to the cloud.

### **Basic Deployment & Frontend Integration:**

- **Model Registration:** Register promising model versions in the Vertex AI Model Registry.
- **Endpoint Deployment (Interactive):** Deploy models to Vertex AI Endpoints directly from a notebook for quick testing of the prediction routine.
- **React Frontend Development:** Develop and iterate on the React UI for anomaly visualization and interaction, sending requests to the deployed Vertex AI Endpoint. This involves basic web development within the Workbench or a local setup.
- **Inference Testing:** Perform test inferences from the notebook and the React frontend to ensure correct data flow and prediction outputs.

#### Model Registration:

**Model Registration in Vertex AI Model Registry**

The Vertex AI Model Registry is a centralized repository to manage the lifecycle of your ML models, including versioning, metadata, and lineage. After a successful training run (local or on Vertex AI), you can register your model.

For this example, we'll assume you have a TensorFlow `SavedModel` artifact. If your training job saved the model to a GCS bucket, you'd point to that GCS path. For local testing, we'll simulate a GCS path to your `model_output` directory.

In [ ]:
import google.cloud.aiplatform as aiplatform
import os
import time

# --- Configuration ---
PROJECT_ID = os.environ.get('GOOGLE_CLOUD_PROJECT', 'your-gcp-project-id') # Replace with your project ID
REGION = 'us-central1' # Or your desired region
MODEL_DISPLAY_NAME = 'my-anomaly-detector-model'
MODEL_DESCRIPTION = 'Anomaly detection model trained with TensorFlow.'
TRAINING_IMAGE_URI = f'{REGION}-docker.pkg.dev/{PROJECT_ID}/vertex-ai-containers/my-tf-trainer:v1'
PREDICTION_IMAGE_URI = f'{REGION}-docker.pkg.dev/{PROJECT_ID}/vertex-ai-containers/my-tf-predictor:v1'

# Initialize Vertex AI SDK
aiplatform.init(project=PROJECT_ID, location=REGION)

print(f"Vertex AI SDK initialized for Project: {PROJECT_ID}, Region: {REGION}")

In [ ]:
from google.cloud import storage

# Assume `model_output/my_model` was generated by your training script (or copied here)
# For local testing, we'll upload this to a GCS bucket.
# In a real Vertex AI Training job, the model would automatically be saved to GCS.

BUCKET_NAME = f'{PROJECT_ID}-vertex-ai-models'
MODEL_SOURCE_DIR = 'model_output/my_model' # Local path to your saved model

# Create a GCS bucket if it doesn't exist
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(BUCKET_NAME)
if not bucket.exists():
    print(f"Creating GCS bucket: gs://{BUCKET_NAME}")
    bucket.create(location=REGION)
else:
    print(f"GCS bucket gs://{BUCKET_NAME} already exists.")

# Upload the SavedModel directory to GCS
GCS_MODEL_PATH = f'gs://{BUCKET_NAME}/models/{MODEL_DISPLAY_NAME}/'
print(f"Uploading model artifacts from {MODEL_SOURCE_DIR} to {GCS_MODEL_PATH}")

def upload_directory_to_gcs(local_dir, bucket_name, gcs_path_prefix):
    """Uploads a local directory recursively to GCS."""
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    for root, _, files in os.walk(local_dir):
        for file in files:
            local_file_path = os.path.join(root, file)
            # Construct the GCS blob name relative to the gcs_path_prefix
            relative_path = os.path.relpath(local_file_path, local_dir)
            blob_name = os.path.join(gcs_path_prefix, relative_path)
            blob = bucket.blob(blob_name)
            blob.upload_from_filename(local_file_path)
            print(f"Uploaded {local_file_path} to gs://{bucket_name}/{blob_name}")

upload_directory_to_gcs(MODEL_SOURCE_DIR, BUCKET_NAME, f'models/{MODEL_DISPLAY_NAME}/')

# Register the model in Vertex AI Model Registry
print("\nRegistering model in Vertex AI Model Registry...")
registered_model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=GCS_MODEL_PATH,
    serving_container_image_uri=PREDICTION_IMAGE_URI,
    description=MODEL_DESCRIPTION,
    # Add optional parameters for custom prediction routine if needed
    # serving_container_predict_route='/predict',
    # serving_container_health_route='/health',
    # serving_container_ports=[8080],
    # serving_container_environment_variables={'MODEL_NAME': 'my_model'},
    # serving_container_explanation_metadata=explain_metadata,
    # serving_container_args=[]
)

print(f"Model registered. Model ID: {registered_model.name}")
print(f"Model Version ID: {registered_model.version_id}")
print(f"Model Resource Name: {registered_model.resource_name}")

# Store the model object for later use
model_obj = registered_model

#### Endpoint Deployment (interactive):

Deploying a model to a Vertex AI Endpoint makes it available for online predictions via a REST API. This section will guide you through interactively deploying your registered model.

In [ ]:
# Create an Endpoint
print("Creating Vertex AI Endpoint...")
endpoint = aiplatform.Endpoint.create(
    display_name=f"{MODEL_DISPLAY_NAME}-endpoint",
    description="Endpoint for anomaly detection model."
)
print(f"Endpoint created. Endpoint ID: {endpoint.name}")
print(f"Endpoint Resource Name: {endpoint.resource_name}")

# Deploy the model to the Endpoint
print(f"\nDeploying model '{model_obj.display_name}' to Endpoint '{endpoint.display_name}'...")
endpoint.deploy(
    model=model_obj,
    deployed_model_display_name=f"{MODEL_DISPLAY_NAME}-deployed",
    machine_type="n1-standard-2", # Choose an appropriate machine type
    min_replica_count=1,
    max_replica_count=1,
    sync=True # Wait for deployment to complete
)

print(f"Model '{model_obj.display_name}' deployed to Endpoint '{endpoint.display_name}' successfully!")
print(f"Endpoint Public DNS: {endpoint.public_endpoint_dns}")

In [ ]:
# Get the deployed model ID
deployed_model_id = endpoint.list_models()[0].id
print(f"Deployed Model ID: {deployed_model_id}")

#### React Frontend Development:

Developing a React frontend directly within the Vertex Workbench Jupyter environment is **not ideal for iterative development**. Jupyter Notebooks are best for data science and ML prototyping, not full-stack web development.

**Recommended Approach for React Frontend:**

1.  **Local Development:** Develop the React app on your local machine using tools like `create-react-app`.
2.  **Separate Repository:** Maintain the frontend code in a separate Git repository.
3.  **Deployment Options:**
    * **Cloud Run:** For a simple, scalable serverless deployment of your React app.
    * **Cloud Hosting (Firebase Hosting, GCS + Load Balancer):** For static site hosting.

**However, for the purpose of demonstrating the integration *conceptually* within this notebook, we'll provide the core JavaScript logic that would interact with your Vertex AI Endpoint.**

**Conceptual Steps for React App (Not runnable in notebook):**

1.  **Project Setup:**
    ```bash
    npx create-react-app anomaly-detector-frontend
    cd anomaly-detector-frontend
    npm start
    ```
2.  **Modify `src/App.js`:**
    * Import `useState`, `useEffect`.
    * Create input fields for data.
    * Implement an API call function to the Vertex AI Endpoint.
    * Display prediction results.

**Key Frontend Interaction Logic (JavaScript - For your `App.js` or similar):**
```javascript
import React, { useState } from 'react';

function App() {
  const [inputData, setInputData] = useState(Array(784).fill(0.5)); // Example: 784 features, initialized with 0.5
  const [prediction, setPrediction] = useState(null);
  const [loading, setLoading] = useState(false);
  const [error, setError] = useState(null);

  // REPLACE WITH YOUR ACTUAL VERTEX AI ENDPOINT DNS
  const VERTEX_AI_ENDPOINT_DNS = "YOUR_VERTEX_AI_ENDPOINT_DNS_HERE"; 
  const PROJECT_ID = "YOUR_GCP_PROJECT_ID";
  const ENDPOINT_ID = "YOUR_VERTEX_AI_ENDPOINT_ID"; // E.g., '1234567890123456789'

  // Function to get an access token for authentication
  // In a real frontend, you'd use a service account key or Firebase Authentication
  // to get a short-lived token server-side or via a secure client-side flow.
  // For simplicity here, we'll assume a way to get a token (e.g., from a backend proxy)
  async function getAccessToken() {
    // This is a placeholder. In production, you'd use a more secure method.
    // E.g., a backend server that fetches a token from Google Metadata Service
    // or uses a service account key.
    // For local dev, you might use 'gcloud auth print-access-token' in a proxy.
    try {
        const response = await fetch('/api/get-access-token'); // Assuming a local proxy server
        const data = await response.json();
        return data.accessToken;
    } catch (err) {
        console.error("Error fetching access token:", err);
        setError("Failed to get authentication token.");
        return null;
    }
  }

  const handlePredict = async () => {
    setLoading(true);
    setError(null);
    setPrediction(null);

    try {
      const accessToken = await getAccessToken();
      if (!accessToken) {
        return;
      }

      const response = await fetch(
        `https://${VERTEX_AI_ENDPOINT_DNS}/v1/projects/${PROJECT_ID}/locations/${REGION}/endpoints/${ENDPOINT_ID}:predict`,
        {
          method: 'POST',
          headers: {
            'Authorization': `Bearer ${accessToken}`,
            'Content-Type': 'application/json',
          },
          body: JSON.stringify({
            instances: [inputData], // Send as an array of instances
          }),
        }
      );

      if (!response.ok) {
        const errorText = await response.text();
        throw new Error(`HTTP error! status: ${response.status}, message: ${errorText}`);
      }

      const data = await response.json();
      setPrediction(data.predictions[0]); // Assuming your model returns a list of predictions
    } catch (e) {
      console.error("Prediction error:", e);
      setError(`Failed to get prediction: ${e.message}`);
    } finally {
      setLoading(false);
    }
  };

  const handleInputChange = (index, value) => {
    const newInputData = [...inputData];
    newInputData[index] = parseFloat(value);
    setInputData(newInputData);
  };

  return (
    <div style={{ padding: '20px', fontFamily: 'Arial, sans-serif' }}>
      <h1>Anomaly Detector</h1>
      <p>Enter data points (e.g., 784 values for an image, or fewer for tabular data):</p>
      {/* Example: A simple input for a few values, or a more complex component for many */}
      <div style={{ marginBottom: '20px' }}>
        {inputData.slice(0, 10).map((value, index) => ( // Display first 10 for simplicity
          <input
            key={index}
            type="number"
            value={value}
            onChange={(e) => handleInputChange(index, e.target.value)}
            style={{ width: '50px', marginRight: '5px' }}
          />
        ))}
        {inputData.length > 10 && <span>...and {inputData.length - 10} more values.</span>}
      </div>

      <button onClick={handlePredict} disabled={loading}>
        {loading ? 'Predicting...' : 'Get Anomaly Prediction'}
      </button>

      {error && <p style={{ color: 'red' }}>Error: {error}</p>}

      {prediction !== null && (
        <div style={{ marginTop: '20px', border: '1px solid #ccc', padding: '15px' }}>
          <h2>Prediction Result:</h2>
          <pre>{JSON.stringify(prediction, null, 2)}</pre>
          {/* Interpret prediction here, e.g., if prediction[0] > threshold, it's an anomaly */}
          {prediction[0] > 0.5 ? 
            <p style={{ color: 'red', fontWeight: 'bold' }}>Likely Anomaly Detected!</p> :
            <p style={{ color: 'green', fontWeight: 'bold' }}>Normal behavior.</p>
          }
        </div>
      )}
      
      <h3>Important Security Note for Frontend Authentication:</h3>
      <p>Directly exposing `gcloud auth print-access-token` results or service account keys in a client-side React app is **highly insecure**. For production, use a secure backend proxy or server-side authentication (e.g., using Firebase Authentication or a custom Node.js/Python backend that fetches tokens on behalf of the client).</p>
    </div>
  );
}

export default App;
```

#### Inference Testing: 

Now that the model is deployed, let's perform some test inferences from both the notebook and conceptually from the React frontend to ensure everything is working as expected.

In [ ]:
import numpy as np
import json

# Generate some dummy input data (e.g., a single instance)
# Ensure the shape and data type match what your model expects
# For a 784-feature input, like flattened MNIST images:
dummy_input_instance = np.random.rand(784).tolist() # Simulate one data point
test_instances = [dummy_input_instance]

print("Performing prediction from notebook...")
try:
    # Use the deployed_model object directly for prediction if you have the endpoint object
    # Or, if starting fresh, use aiplatform.Endpoint(endpoint_resource_name)
    
    predictions = endpoint.predict(instances=test_instances)
    
    print("\nPrediction Response:")
    print(json.dumps(predictions.predictions, indent=2))
    
    # You can add logic here to interpret the prediction
    # For example, if it's a binary classification for anomaly:
    if predictions.predictions and len(predictions.predictions[0]) > 0:
        if predictions.predictions[0][0] > 0.5: # Assuming output is probability of anomaly
            print("Model predicted: Likely Anomaly")
        else:
            print("Model predicted: Normal")
    else:
        print("Unexpected prediction format.")

except Exception as e:
    print(f"Error during prediction: {e}")

### Conceptual Inference Testing from React Frontend

As described in Section 3, the `handlePredict` function in the `App.js` example demonstrates how the React frontend would send prediction requests to your Vertex AI Endpoint.

**To actually test this:**
1.  Set up your React application locally (outside of this Workbench notebook).
2.  Replace the placeholder `YOUR_VERTEX_AI_ENDPOINT_DNS_HERE`, `YOUR_GCP_PROJECT_ID`, and `YOUR_VERTEX_AI_ENDPOINT_ID` with the actual values obtained from this notebook (e.g., from Cell 18).
3.  Implement a secure way to get an access token for authentication (e.g., a simple Node.js proxy server that uses `gcloud auth print-access-token` for local development, or a proper service account key in a backend for production).
4.  Run your React application (`npm start`).
5.  Interact with the UI, input data, and click the "Get Anomaly Prediction" button. Observe the network requests and the displayed prediction results.

## Clean Up (Optional but Recommended)

To avoid incurring unnecessary costs, it's good practice to undeploy models and delete endpoints when they are no longer needed.

In [ ]:
# WARNING: This will undeploy your model and delete the endpoint.
# Only run this cell when you are finished with testing.

print(f"Undeploying model from endpoint: {endpoint.display_name}...")
endpoint.undeploy_all()
print("Model undeployed.")

print(f"Deleting endpoint: {endpoint.display_name}...")
endpoint.delete()
print("Endpoint deleted.")

# You can also delete the model from the registry if desired
# print(f"Deleting model from registry: {model_obj.display_name}...")
# model_obj.delete()
# print("Model deleted from registry.")

# To delete the GCS bucket:
# storage_client.delete_bucket(BUCKET_NAME)
# print(f"Deleted GCS bucket: {BUCKET_NAME}")

----